In [1]:
import toyplot
import toytree
import pandas as pd

Generate trees
=====
For each scaffold, generate a tree with renamed tips.

In [3]:
# Load sample metadata to be used with all trees.
DATA = pd.read_csv("../oak-data/Hipp-2019-table-S1.csv")
columns = [
    "SRA_Run_Accession_RADseq", 
    "subgenus", 
    "section", 
    "clade", 
    "Cleaned_NAMES-USE-THIS",
]
data = DATA.loc[:, columns]
data.columns = ["SRA", "subgenus", "section", "clade", "species"]
data.head()

,SRA,subgenus,section,clade,species
0,SRR5632368,subg. Quercus,Quercus,Leucomexicana,Quercus rugosa
1,SRR5632418,subg. Quercus,Lobatae,Erythromexicana,Quercus calophylla
2,SRR5632456,subg. Quercus,Quercus,Leucomexicana,Quercus magnoliifolia
3,SRR5632554,subg. Quercus,Lobatae,Erythromexicana,Quercus crassipes
4,SRR5632551,subg. Quercus,Lobatae,Erythromexicana,Quercus scytophylla


Scaffold 01

In [3]:
tre01 = toytree.tree("/moto/eaton/users/hnl2109/analysis-raxml/RAxML_bipartitions.basic_scaffold01")

# Include only samples in the tree.
mask01 = [i in tre01.get_tip_labels() for i in data.SRA]
tipdata01 = data.loc[mask01, :]
tipdata01 = tipdata01.reset_index(drop=True)
tipdata01.head()

,SRA,subgenus,section,clade,species
0,SRR5632418,subg. Quercus,Lobatae,Erythromexicana,Quercus calophylla
1,SRR5632552,subg. Quercus,Lobatae,Erythromexicana,Quercus crassipes
2,SRR5284351,subg. Quercus,Quercus,Albae,Quercus michauxii
3,SRR5632579,subg. Quercus,Lobatae,Laurifoliae,Quercus pagoda
4,SRR5284355,subg. Quercus,Quercus,Prinoids,Quercus macrocarpa


In [4]:
# Set missing values to empty.
tipdata01.loc[tipdata01.subgenus.isna(), ["subgenus", "section", "clade"]] = ""

In [5]:
# Convert SRA tags to complex tip description (subgenus, section, clade, species), saved as a dict.
tips_dict01 = {}

# The attribute .shape refers to the dimensions of an array (rows, columns).  .shape[0] iterates through the number of rows
# in the dataset.
for idx in range(tipdata01.shape[0]):
    
    # For outgroups, we are only interested in the species.
    if tipdata01.subgenus[idx] == "" :
        subg = sect = clad = "NaN"
        spec = tipdata01.species[idx].replace(" ", "-")
        
    # For oaks, get all the data.
    else:
        subg = tipdata01.subgenus[idx].split()[-1]
        sect = tipdata01.section[idx]
        clad = tipdata01.clade[idx]
        spec = tipdata01.species[idx].split()[-1]
    
    # For each SRA tag, input the values just determined, and save the SRA/stuff key/value pair to the dict.
    if tipdata01.subgenus[idx] == "" :
        tips_dict01[tipdata01.SRA[idx]] = "{}|{}|{}|{}".format(subg, sect, clad, spec)
    else:
        tips_dict01[tipdata01.SRA[idx]] = "{}|{}|{}|Q.{}".format(subg, sect, clad, spec)

In [6]:
# The attribute .set_node_values can be used to modify node attributes.  The first argument picks the parameter to be
# modified, in this case the tip names.  The second argument requires a dict of {node: value}.  For each tip where the name
# (as a string) matches the node, the value is substituted instead.  Here this is saved to a new tree variable.

ntre01 = tre01.set_node_values('name', tips_dict01)

In [7]:
# Draw a rooted tree.
rtre01 = ntre01.root(wildcard="Lithocarpus")
canvas01, axes01, mark01 = rtre01.draw(
    height=2000,
    tip_labels_style={"font-size": "8px"},
    use_edge_lengths=False,
    
    # Color-code the tree according to clades defined by Hipp et al.
    edge_colors=rtre01.get_edge_values_mapped({
        rtre01.get_mrca_idx_from_tip_labels(wildcard="Cerris|"),
        rtre01.get_mrca_idx_from_tip_labels(wildcard="Lobatae|"),
        rtre01.get_mrca_idx_from_tip_labels(wildcard="Roburoids|"),
        rtre01.get_mrca_idx_from_tip_labels(wildcard="Virentes|"),
        rtre01.get_mrca_idx_from_tip_labels(wildcard="Protoba"),
        rtre01.get_mrca_idx_from_tip_labels(wildcard="Leuco"),
        rtre01.get_mrca_idx_from_tip_labels(wildcard="Ponticae|"),
    }),
)

<svg class="toyplot-canvas-Canvas" height="2000.0px" id="tc3c2846e1e3849e987dd017faf9d2f18" preserveAspectRatio="xMidYMid meet" style="background-color:transparent;border-color:#292724;border-style:none;border-width:1.0;fill:rgb(16.1%,15.3%,14.1%);fill-opacity:1.0;font-family:Helvetica;font-size:12px;opacity:1.0;stroke:rgb(16.1%,15.3%,14.1%);stroke-opacity:1.0;stroke-width:1.0" viewBox="0 0 500.0 2000.0" width="500.0px" xmlns="http://www.w3.org/2000/svg" xmlns:toyplot="http://www.sandia.gov/toyplot" xmlns:xlink="http://www.w3.org/1999/xlink"> Quercus|Quercus|Leucomexicana|Q.turbinella Quercus|Quercus|Leucomexicana|Q.toumeyi Quercus|Quercus|Leucomexicana|Q.ajoensis Quercus|Quercus|Leucomexicana|Q.grisea Quercus|Quercus|Leucomexicana|Q.arizonica Quercus|Quercus|Leucomexicana|Q.glaucoides Quercus|Quercus|Leucomexicana|Q.chihuahuensis Quercus|Quercus|Leucomexicana|Q.oblongifolia Quercus|Quercus|Leucomexicana|Q.laeta Quercus|Quercus|Leucomexicana|Q.laeta Quercus|Quercus|Leucomexicana|Q.diversifolia Quercus|Quercus|Leucomexicana|Q.arizonica Quercus|Quercus|Leucomexicana|Q.obtusata Quercus|Quercus|Leucomexicana|Q.deserticola Quercus|Quercus|Leucomexicana|Q.nudinervis Quercus|Quercus|Leucomexicana|Q.resinosa Quercus|Quercus|Leucomexicana|Q.magnoliifolia Quercus|Quercus|Leucomexicana|Q.subspathulata Quercus|Quercus|Leucomexicana|Q.segoviensis Quercus|Quercus|Leucomexicana|Q.purulhana Quercus|Quercus|Leucomexicana|Q.species Quercus|Quercus|Leucomexicana|Q.liebmanii Quercus|Quercus|Leucomexicana|Q.glaucescens Quercus|Quercus|Leucomexicana|Q.peduncularis Quercus|Quercus|Leucomexicana|Q.potosina Quercus|Quercus|Leucomexicana|Q.striatula Quercus|Quercus|Leucomexicana|Q.greggii Quercus|Quercus|Leucomexicana|Q.martinezii Quercus|Quercus|Leucomexicana|Q.rugosa Quercus|Quercus|Leucomexicana|Q.glabrescens Quercus|Quercus|Texas white oaks|Q.vaseyana Quercus|Quercus|Texas white oaks|Q.pungens Quercus|Quercus|Texas white oaks|Q.hinckleyi Quercus|Quercus|Stellatae|Q.laceyi Quercus|Quercus|Texas white oaks|Q.mohriana Quercus|Quercus|Texas white oaks|Q.polymorpha Quercus|Quercus|Leucomexicana|Q.insignis Quercus|Quercus|Leucomexicana|Q.corrugata Quercus|Quercus|Leucomexicana|Q.copeyensis Quercus|Quercus|Leucomexicana|Q.lancifolia Quercus|Quercus|Leucomexicana|Q.germana Quercus|Quercus|Stellatae|Q.margarettae Quercus|Quercus|Stellatae|Q.havardii Quercus|Quercus|Stellatae|Q.similis Quercus|Quercus|Stellatae|Q.stellata Quercus|Quercus|Stellatae|Q.chapmanii Quercus|Quercus|Stellatae|Q.boyntonii Quercus|Quercus|Stellatae|Q.austrina Quercus|Quercus|Stellatae|Q.oglethorpensis Quercus|Quercus|Stellatae|Q.sinuata Quercus|Quercus|Prinoids|Q.prinoides Quercus|Quercus|Prinoids|Q.muehlenbergii Quercus|Quercus|Albae|Q.alba Quercus|Quercus|Albae|Q.michauxii Quercus|Quercus|Albae|Q.montana Quercus|Quercus|Dumosae|Q.lobata Quercus|Quercus|Prinoids|Q.macrocarpa Quercus|Quercus|Prinoids|Q.lyrata Quercus|Quercus|Prinoids|Q.bicolor Quercus|Quercus|Roburoids|Q.infectoria Quercus|Quercus|Roburoids|Q.boissieri Quercus|Quercus|Roburoids|Q.cedrorum Quercus|Quercus|Roburoids|Q.kotschyana Quercus|Quercus|Roburoids|Q.pubescens Quercus|Quercus|Roburoids|Q.faginea Quercus|Quercus|Roburoids|Q.frainetto Quercus|Quercus|Roburoids|Q.pyrenaica Quercus|Quercus|Roburoids|Q.macranthera Quercus|Quercus|Roburoids|Q.petraea Quercus|Quercus|Roburoids|Q.dalechampii Quercus|Quercus|Roburoids|Q.vulcanica Quercus|Quercus|Roburoids|Q.lusitanica Quercus|Quercus|Roburoids|Q.robur reference Quercus|Quercus|Roburoids|Q.hartwissiana Quercus|Quercus|Roburoids|Q.imeretina Quercus|Quercus|Roburoids|Q.canariensis Quercus|Quercus|Roburoids|Q.aliena Quercus|Quercus|Roburoids|Q.griffithii Quercus|Quercus|Roburoids|Q.fabri Quercus|Quercus|Roburoids|Q.dentata Quercus|Quercus|Roburoids|Q.yunnanensis Quercus|Quercus|Roburoids|Q.serrata Quercus|Quercus|Roburoids|Q.mongolica Quercus|Quercus|Dumosae|Q.dumosa Quercus|Quercus|Dumosae|Q.pacifica Quercus|Quercus|Dumosae|Q.douglasii Quercus|Quercus|Dumosae|Q.john-tuckeri Q

In [9]:
tre00 = toytree.tree("/moto/eaton/users/hnl2109/analysis-raxml/RAxML_bipartitions.test50")
mask00 = [i in tre00.get_tip_labels() for i in data.SRA]
tipdata00 = data.loc[mask00, :]
tipdata00 = tipdata00.reset_index(drop=True)
tipdata00.loc[tipdata00.subgenus.isna(), ["subgenus", "section", "clade"]] = ""
tips_dict00 = {}
for idx in range(tipdata00.shape[0]):
    if tipdata00.subgenus[idx] == "" :
        subg = sect = clad = "NaN"
        spec = tipdata00.species[idx].replace(" ", "-")
    else:
        subg = tipdata00.subgenus[idx].split()[-1]
        sect = tipdata00.section[idx]
        clad = tipdata00.clade[idx]
        spec = tipdata00.species[idx].split()[-1]
    if tipdata00.subgenus[idx] == "" :
        tips_dict00[tipdata00.SRA[idx]] = "{}|{}|{}|{}".format(subg, sect, clad, spec)
    else:
        tips_dict00[tipdata00.SRA[idx]] = "{}|{}|{}|Q.{}".format(subg, sect, clad, spec)
ntre00 = tre00.set_node_values('name', tips_dict00)
rtre00 = ntre00.root(wildcard="Lithocarpus")
canvas00, axes00, mark00 = rtre00.draw(
    height=2000,
    tip_labels_style={"font-size": "8px"},
    use_edge_lengths=False,
    edge_colors=rtre02.get_edge_values_mapped({
        rtre02.get_mrca_idx_from_tip_labels(wildcard="Cerris|"),
        rtre02.get_mrca_idx_from_tip_labels(wildcard="Lobatae|"),
        rtre02.get_mrca_idx_from_tip_labels(wildcard="Roburoids|"),
        rtre02.get_mrca_idx_from_tip_labels(wildcard="Virentes|"),
        rtre02.get_mrca_idx_from_tip_labels(wildcard="Protoba"),
        rtre02.get_mrca_idx_from_tip_labels(wildcard="Leuco"),
        rtre02.get_mrca_idx_from_tip_labels(wildcard="Ponticae|"),
    }),
)

<svg class="toyplot-canvas-Canvas" height="2000.0px" id="t1705426e9a2845399e2e274ff5dd1e5b" preserveAspectRatio="xMidYMid meet" style="background-color:transparent;border-color:#292724;border-style:none;border-width:1.0;fill:rgb(16.1%,15.3%,14.1%);fill-opacity:1.0;font-family:Helvetica;font-size:12px;opacity:1.0;stroke:rgb(16.1%,15.3%,14.1%);stroke-opacity:1.0;stroke-width:1.0" viewBox="0 0 500.0 2000.0" width="500.0px" xmlns="http://www.w3.org/2000/svg" xmlns:toyplot="http://www.sandia.gov/toyplot" xmlns:xlink="http://www.w3.org/1999/xlink"> Quercus|Quercus|Leucomexicana|Q.turbinella Quercus|Quercus|Leucomexicana|Q.ajoensis Quercus|Quercus|Leucomexicana|Q.toumeyi Quercus|Quercus|Leucomexicana|Q.oblongifolia Quercus|Quercus|Leucomexicana|Q.chihuahuensis Quercus|Quercus|Leucomexicana|Q.laeta Quercus|Quercus|Leucomexicana|Q.laeta Quercus|Quercus|Leucomexicana|Q.arizonica Quercus|Quercus|Leucomexicana|Q.diversifolia Quercus|Quercus|Leucomexicana|Q.potosina Quercus|Quercus|Leucomexicana|Q.peduncularis Quercus|Quercus|Leucomexicana|Q.obtusata Quercus|Quercus|Leucomexicana|Q.deserticola Quercus|Quercus|Leucomexicana|Q.arizonica Quercus|Quercus|Leucomexicana|Q.grisea Quercus|Quercus|Leucomexicana|Q.glaucoides Quercus|Quercus|Leucomexicana|Q.engelmannii Quercus|Quercus|Leucomexicana|Q.copeyensis Quercus|Quercus|Leucomexicana|Q.lancifolia Quercus|Quercus|Leucomexicana|Q.insignis Quercus|Quercus|Leucomexicana|Q.corrugata Quercus|Quercus|Leucomexicana|Q.germana Quercus|Quercus|Leucomexicana|Q.glabrescens Quercus|Quercus|Leucomexicana|Q.martinezii Quercus|Quercus|Leucomexicana|Q.rugosa Quercus|Quercus|Leucomexicana|Q.greggii Quercus|Quercus|Leucomexicana|Q.striatula Quercus|Quercus|Leucomexicana|Q.magnoliifolia Quercus|Quercus|Leucomexicana|Q.resinosa Quercus|Quercus|Leucomexicana|Q.nudinervis Quercus|Quercus|Leucomexicana|Q.liebmanii Quercus|Quercus|Leucomexicana|Q.subspathulata Quercus|Quercus|Leucomexicana|Q.segoviensis Quercus|Quercus|Leucomexicana|Q.purulhana Quercus|Quercus|Leucomexicana|Q.species Quercus|Quercus|Leucomexicana|Q.glaucescens Quercus|Quercus|Stellatae|Q.margarettae Quercus|Quercus|Stellatae|Q.chapmanii Quercus|Quercus|Stellatae|Q.similis Quercus|Quercus|Stellatae|Q.austrina Quercus|Quercus|Stellatae|Q.stellata Quercus|Quercus|Stellatae|Q.boyntonii Quercus|Quercus|Stellatae|Q.havardii Quercus|Quercus|Stellatae|Q.oglethorpensis Quercus|Quercus|Stellatae|Q.sinuata Quercus|Quercus|Texas white oaks|Q.hinckleyi Quercus|Quercus|Texas white oaks|Q.pungens Quercus|Quercus|Texas white oaks|Q.vaseyana Quercus|Quercus|Texas white oaks|Q.polymorpha Quercus|Quercus|Texas white oaks|Q.mohriana Quercus|Quercus|Stellatae|Q.laceyi Quercus|Quercus|Roburoids|Q.infectoria Quercus|Quercus|Roburoids|Q.boissieri Quercus|Quercus|Roburoids|Q.cedrorum Quercus|Quercus|Roburoids|Q.vulcanica Quercus|Quercus|Roburoids|Q.frainetto Quercus|Quercus|Roburoids|Q.canariensis reference Quercus|Quercus|Roburoids|Q.imeretina Quercus|Quercus|Roburoids|Q.pyrenaica Quercus|Quercus|Roburoids|Q.dalechampii Quercus|Quercus|Roburoids|Q.pubescens Quercus|Quercus|Roburoids|Q.hartwissiana Quercus|Quercus|Roburoids|Q.robur Quercus|Quercus|Roburoids|Q.lusitanica Quercus|Quercus|Roburoids|Q.petraea Quercus|Quercus|Roburoids|Q.macranthera Quercus|Quercus|Roburoids|Q.faginea Quercus|Quercus|Roburoids|Q.kotschyana Quercus|Quercus|Roburoids|Q.fabri Quercus|Quercus|Roburoids|Q.griffithii Quercus|Quercus|Roburoids|Q.yunnanensis Quercus|Quercus|Roburoids|Q.aliena Quercus|Quercus|Roburoids|Q.serrata Quercus|Quercus|Roburoids|Q.dentata Quercus|Quercus|Roburoids|Q.mongolica Quercus|Quercus|Dumosae|Q.douglasii Quercus|Quercus|Dumosae|Q.john-tuckeri Quercus|Quercus|Dumosae|Q.pacifica Quercus|Quercus|Dumosae|Q.cornelius-mulleri Quercus|Quercus|Dumosae|Q.dumosa Quercus|Quercus|Dumosae|Q.durata Quercus|Quercus|Dumosae|Q.berberidifolia Quercus|Quercus|Dumosae|Q.garryana Quercus|Quercus|Dumosae|Q.lobata Quercus|Quercus|Prinoids|Q.bicolor Quercus|Quercus|Prinoids|Q.lyrata Quercus|Quercus

In [8]:
tre02 = toytree.tree("/moto/eaton/users/hnl2109/analysis-raxml/RAxML_bipartitions.basic_scaffold02")
mask02 = [i in tre02.get_tip_labels() for i in data.SRA]
tipdata02 = data.loc[mask02, :]
tipdata02 = tipdata02.reset_index(drop=True)
tipdata02.loc[tipdata02.subgenus.isna(), ["subgenus", "section", "clade"]] = ""
tips_dict02 = {}
for idx in range(tipdata02.shape[0]):
    if tipdata02.subgenus[idx] == "" :
        subg = sect = clad = "NaN"
        spec = tipdata02.species[idx].replace(" ", "-")
    else:
        subg = tipdata02.subgenus[idx].split()[-1]
        sect = tipdata02.section[idx]
        clad = tipdata02.clade[idx]
        spec = tipdata02.species[idx].split()[-1]
    if tipdata02.subgenus[idx] == "" :
        tips_dict02[tipdata02.SRA[idx]] = "{}|{}|{}|{}".format(subg, sect, clad, spec)
    else:
        tips_dict02[tipdata02.SRA[idx]] = "{}|{}|{}|Q.{}".format(subg, sect, clad, spec)
ntre02 = tre02.set_node_values('name', tips_dict02)
rtre02 = ntre02.root(wildcard="Lithocarpus")
canvas02, axes02, mark02 = rtre02.draw(
    height=2000,
    tip_labels_style={"font-size": "8px"},
    use_edge_lengths=False,
    edge_colors=rtre02.get_edge_values_mapped({
        rtre02.get_mrca_idx_from_tip_labels(wildcard="Cerris|"),
        rtre02.get_mrca_idx_from_tip_labels(wildcard="Lobatae|"),
        rtre02.get_mrca_idx_from_tip_labels(wildcard="Roburoids|"),
        rtre02.get_mrca_idx_from_tip_labels(wildcard="Virentes|"),
        rtre02.get_mrca_idx_from_tip_labels(wildcard="Protoba"),
        rtre02.get_mrca_idx_from_tip_labels(wildcard="Leuco"),
        rtre02.get_mrca_idx_from_tip_labels(wildcard="Ponticae|"),
    }),
)

<svg class="toyplot-canvas-Canvas" height="2000.0px" id="t8369fbf56f4540d1b935b79fb9dfeabc" preserveAspectRatio="xMidYMid meet" style="background-color:transparent;border-color:#292724;border-style:none;border-width:1.0;fill:rgb(16.1%,15.3%,14.1%);fill-opacity:1.0;font-family:Helvetica;font-size:12px;opacity:1.0;stroke:rgb(16.1%,15.3%,14.1%);stroke-opacity:1.0;stroke-width:1.0" viewBox="0 0 500.0 2000.0" width="500.0px" xmlns="http://www.w3.org/2000/svg" xmlns:toyplot="http://www.sandia.gov/toyplot" xmlns:xlink="http://www.w3.org/1999/xlink"> Quercus|Quercus|Leucomexicana|Q.arizonica Quercus|Quercus|Leucomexicana|Q.oblongifolia Quercus|Quercus|Leucomexicana|Q.turbinella Quercus|Quercus|Leucomexicana|Q.ajoensis Quercus|Quercus|Leucomexicana|Q.grisea Quercus|Quercus|Leucomexicana|Q.chihuahuensis Quercus|Quercus|Leucomexicana|Q.laeta Quercus|Quercus|Leucomexicana|Q.greggii Quercus|Quercus|Leucomexicana|Q.toumeyi Quercus|Quercus|Leucomexicana|Q.striatula Quercus|Quercus|Leucomexicana|Q.engelmannii Quercus|Quercus|Leucomexicana|Q.laeta Quercus|Quercus|Leucomexicana|Q.glaucoides Quercus|Quercus|Leucomexicana|Q.arizonica Quercus|Quercus|Leucomexicana|Q.obtusata Quercus|Quercus|Leucomexicana|Q.peduncularis Quercus|Quercus|Leucomexicana|Q.potosina Quercus|Quercus|Leucomexicana|Q.deserticola Quercus|Quercus|Leucomexicana|Q.species Quercus|Quercus|Leucomexicana|Q.resinosa Quercus|Quercus|Leucomexicana|Q.subspathulata Quercus|Quercus|Leucomexicana|Q.nudinervis Quercus|Quercus|Leucomexicana|Q.segoviensis Quercus|Quercus|Leucomexicana|Q.purulhana Quercus|Quercus|Leucomexicana|Q.liebmanii Quercus|Quercus|Leucomexicana|Q.magnoliifolia Quercus|Quercus|Leucomexicana|Q.glabrescens Quercus|Quercus|Leucomexicana|Q.martinezii Quercus|Quercus|Leucomexicana|Q.rugosa Quercus|Quercus|Leucomexicana|Q.diversifolia Quercus|Quercus|Leucomexicana|Q.corrugata Quercus|Quercus|Leucomexicana|Q.insignis Quercus|Quercus|Leucomexicana|Q.copeyensis Quercus|Quercus|Leucomexicana|Q.lancifolia Quercus|Quercus|Leucomexicana|Q.germana Quercus|Quercus|Leucomexicana|Q.glaucescens Quercus|Quercus|Stellatae|Q.margarettae Quercus|Quercus|Stellatae|Q.havardii Quercus|Quercus|Stellatae|Q.oglethorpensis Quercus|Quercus|Stellatae|Q.similis Quercus|Quercus|Stellatae|Q.austrina Quercus|Quercus|Stellatae|Q.stellata Quercus|Quercus|Stellatae|Q.boyntonii Quercus|Quercus|Stellatae|Q.chapmanii Quercus|Quercus|Stellatae|Q.sinuata Quercus|Quercus|Texas white oaks|Q.pungens Quercus|Quercus|Texas white oaks|Q.hinckleyi Quercus|Quercus|Texas white oaks|Q.vaseyana Quercus|Quercus|Stellatae|Q.laceyi Quercus|Quercus|Texas white oaks|Q.polymorpha Quercus|Quercus|Texas white oaks|Q.mohriana Quercus|Quercus|Roburoids|Q.hartwissiana reference Quercus|Quercus|Roburoids|Q.imeretina Quercus|Quercus|Roburoids|Q.canariensis Quercus|Quercus|Roburoids|Q.robur Quercus|Quercus|Roburoids|Q.lusitanica Quercus|Quercus|Roburoids|Q.pyrenaica Quercus|Quercus|Roburoids|Q.petraea Quercus|Quercus|Roburoids|Q.dalechampii Quercus|Quercus|Roburoids|Q.faginea Quercus|Quercus|Roburoids|Q.vulcanica Quercus|Quercus|Roburoids|Q.macranthera Quercus|Quercus|Roburoids|Q.infectoria Quercus|Quercus|Roburoids|Q.boissieri Quercus|Quercus|Roburoids|Q.cedrorum Quercus|Quercus|Roburoids|Q.kotschyana Quercus|Quercus|Roburoids|Q.pubescens Quercus|Quercus|Roburoids|Q.frainetto Quercus|Quercus|Roburoids|Q.yunnanensis Quercus|Quercus|Roburoids|Q.dentata Quercus|Quercus|Roburoids|Q.mongolica Quercus|Quercus|Roburoids|Q.griffithii Quercus|Quercus|Roburoids|Q.fabri Quercus|Quercus|Roburoids|Q.aliena Quercus|Quercus|Roburoids|Q.serrata Quercus|Quercus|Dumosae|Q.lobata Quercus|Quercus|Dumosae|Q.john-tuckeri Quercus|Quercus|Dumosae|Q.douglasii Quercus|Quercus|Dumosae|Q.pacifica Quercus|Quercus|Dumosae|Q.dumosa Quercus|Quercus|Dumosae|Q.cornelius-mulleri Quercus|Quercus|Dumosae|Q.durata Quercus|Quercus|Dumosae|Q.berberidifolia Quercus|Quercus|Dumosae|Q.garryana Quercus|Quercus|Albae|Q.alba Quercus|Quercus|Albae|Q.michauxii Quercus|Quercus|Albae

In [10]:
tre03 = toytree.tree("/moto/eaton/users/hnl2109/analysis-raxml/RAxML_bipartitions.basic_scaffold03")
mask03 = [i in tre03.get_tip_labels() for i in data.SRA]
tipdata03 = data.loc[mask03, :]
tipdata03 = tipdata03.reset_index(drop=True)
tipdata03.loc[tipdata03.subgenus.isna(), ["subgenus", "section", "clade"]] = ""
tips_dict03 = {}
for idx in range(tipdata03.shape[0]):
    if tipdata03.subgenus[idx] == "" :
        subg = sect = clad = "NaN"
        spec = tipdata03.species[idx].replace(" ", "-")
    else:
        subg = tipdata03.subgenus[idx].split()[-1]
        sect = tipdata03.section[idx]
        clad = tipdata03.clade[idx]
        spec = tipdata03.species[idx].split()[-1]
    if tipdata03.subgenus[idx] == "" :
        tips_dict03[tipdata03.SRA[idx]] = "{}|{}|{}|{}".format(subg, sect, clad, spec)
    else:
        tips_dict03[tipdata03.SRA[idx]] = "{}|{}|{}|Q.{}".format(subg, sect, clad, spec)
ntre03 = tre03.set_node_values('name', tips_dict03)
rtre03 = ntre03.root(wildcard="Lithocarpus")
canvas03, axes03, mark03 = rtre03.draw(
    height=2000,
    tip_labels_style={"font-size": "8px"},
    use_edge_lengths=False,
    edge_colors=rtre03.get_edge_values_mapped({
        rtre03.get_mrca_idx_from_tip_labels(wildcard="Cerris|"),
        rtre03.get_mrca_idx_from_tip_labels(wildcard="Lobatae|"),
        rtre03.get_mrca_idx_from_tip_labels(wildcard="Roburoids|"),
        rtre03.get_mrca_idx_from_tip_labels(wildcard="Virentes|"),
        rtre03.get_mrca_idx_from_tip_labels(wildcard="Protoba"),
        rtre03.get_mrca_idx_from_tip_labels(wildcard="Leuco"),
        rtre03.get_mrca_idx_from_tip_labels(wildcard="Ponticae|"),
    }),
)

<svg class="toyplot-canvas-Canvas" height="2000.0px" id="t229c781805db40b5bc155fd4b8f3389c" preserveAspectRatio="xMidYMid meet" style="background-color:transparent;border-color:#292724;border-style:none;border-width:1.0;fill:rgb(16.1%,15.3%,14.1%);fill-opacity:1.0;font-family:Helvetica;font-size:12px;opacity:1.0;stroke:rgb(16.1%,15.3%,14.1%);stroke-opacity:1.0;stroke-width:1.0" viewBox="0 0 500.0 2000.0" width="500.0px" xmlns="http://www.w3.org/2000/svg" xmlns:toyplot="http://www.sandia.gov/toyplot" xmlns:xlink="http://www.w3.org/1999/xlink"> Quercus|Quercus|Leucomexicana|Q.insignis Quercus|Quercus|Leucomexicana|Q.corrugata Quercus|Quercus|Leucomexicana|Q.copeyensis Quercus|Quercus|Leucomexicana|Q.glabrescens Quercus|Quercus|Leucomexicana|Q.glaucescens Quercus|Quercus|Leucomexicana|Q.obtusata Quercus|Quercus|Leucomexicana|Q.martinezii Quercus|Quercus|Leucomexicana|Q.rugosa Quercus|Quercus|Leucomexicana|Q.liebmanii Quercus|Quercus|Leucomexicana|Q.subspathulata Quercus|Quercus|Leucomexicana|Q.resinosa Quercus|Quercus|Leucomexicana|Q.nudinervis Quercus|Quercus|Leucomexicana|Q.potosina Quercus|Quercus|Leucomexicana|Q.striatula Quercus|Quercus|Leucomexicana|Q.arizonica Quercus|Quercus|Leucomexicana|Q.species Quercus|Quercus|Leucomexicana|Q.greggii Quercus|Quercus|Leucomexicana|Q.glaucoides Quercus|Quercus|Leucomexicana|Q.diversifolia Quercus|Quercus|Leucomexicana|Q.engelmannii Quercus|Quercus|Leucomexicana|Q.toumeyi Quercus|Quercus|Leucomexicana|Q.oblongifolia Quercus|Quercus|Leucomexicana|Q.deserticola Quercus|Quercus|Leucomexicana|Q.laeta Quercus|Quercus|Leucomexicana|Q.laeta Quercus|Quercus|Leucomexicana|Q.ajoensis Quercus|Quercus|Leucomexicana|Q.peduncularis Quercus|Quercus|Leucomexicana|Q.grisea Quercus|Quercus|Leucomexicana|Q.chihuahuensis Quercus|Quercus|Leucomexicana|Q.segoviensis Quercus|Quercus|Leucomexicana|Q.arizonica Quercus|Quercus|Leucomexicana|Q.purulhana Quercus|Quercus|Leucomexicana|Q.magnoliifolia Quercus|Quercus|Leucomexicana|Q.turbinella Quercus|Quercus|Texas white oaks|Q.vaseyana Quercus|Quercus|Texas white oaks|Q.hinckleyi Quercus|Quercus|Texas white oaks|Q.pungens Quercus|Quercus|Stellatae|Q.laceyi Quercus|Quercus|Stellatae|Q.sinuata Quercus|Quercus|Leucomexicana|Q.lancifolia Quercus|Quercus|Texas white oaks|Q.polymorpha Quercus|Quercus|Leucomexicana|Q.germana Quercus|Quercus|Texas white oaks|Q.mohriana Quercus|Quercus|Stellatae|Q.boyntonii Quercus|Quercus|Stellatae|Q.margarettae Quercus|Quercus|Stellatae|Q.austrina Quercus|Quercus|Stellatae|Q.chapmanii Quercus|Quercus|Stellatae|Q.similis Quercus|Quercus|Stellatae|Q.stellata Quercus|Quercus|Stellatae|Q.havardii Quercus|Quercus|Stellatae|Q.oglethorpensis Quercus|Quercus|Albae|Q.alba Quercus|Quercus|Albae|Q.montana Quercus|Quercus|Albae|Q.michauxii Quercus|Quercus|Prinoids|Q.muehlenbergii Quercus|Quercus|Prinoids|Q.prinoides Quercus|Quercus|Prinoids|Q.bicolor Quercus|Quercus|Prinoids|Q.lyrata Quercus|Quercus|Prinoids|Q.macrocarpa Quercus|Quercus|Dumosae|Q.lobata Quercus|Quercus|Roburoids|Q.petraea Quercus|Quercus|Roburoids|Q.dalechampii Quercus|Quercus|Roburoids|Q.lusitanica Quercus|Quercus|Roburoids|Q.boissieri Quercus|Quercus|Roburoids|Q.infectoria Quercus|Quercus|Roburoids|Q.cedrorum Quercus|Quercus|Roburoids|Q.pubescens Quercus|Quercus|Roburoids|Q.macranthera Quercus|Quercus|Roburoids|Q.imeretina Quercus|Quercus|Roburoids|Q.robur reference Quercus|Quercus|Roburoids|Q.canariensis Quercus|Quercus|Roburoids|Q.hartwissiana Quercus|Quercus|Roburoids|Q.pyrenaica Quercus|Quercus|Roburoids|Q.kotschyana Quercus|Quercus|Roburoids|Q.faginea Quercus|Quercus|Roburoids|Q.vulcanica Quercus|Quercus|Roburoids|Q.frainetto Quercus|Quercus|Roburoids|Q.aliena Quercus|Quercus|Roburoids|Q.fabri Quercus|Quercus|Roburoids|Q.yunnanensis Quercus|Quercus|Roburoids|Q.dentata Quercus|Quercus|Roburoids|Q.serrata Quercus|Quercus|Roburoids|Q.mongolica Quercus|Quercus|Roburoids|Q.griffithii Quercus|Quercus|Dumosae|Q.john-tuckeri Quercus|Quercus|Dumosae|Q.pacifica Quercus|Quercus|Dumosae|Q.

In [11]:
tre04 = toytree.tree("/moto/eaton/users/hnl2109/analysis-raxml/RAxML_bipartitions.basic_scaffold04")
mask04 = [i in tre04.get_tip_labels() for i in data.SRA]
tipdata04 = data.loc[mask04, :]
tipdata04 = tipdata04.reset_index(drop=True)
tipdata04.loc[tipdata04.subgenus.isna(), ["subgenus", "section", "clade"]] = ""
tips_dict04 = {}
for idx in range(tipdata04.shape[0]):
    if tipdata04.subgenus[idx] == "" :
        subg = sect = clad = "NaN"
        spec = tipdata04.species[idx].replace(" ", "-")
    else:
        subg = tipdata04.subgenus[idx].split()[-1]
        sect = tipdata04.section[idx]
        clad = tipdata04.clade[idx]
        spec = tipdata04.species[idx].split()[-1]
    if tipdata04.subgenus[idx] == "" :
        tips_dict04[tipdata04.SRA[idx]] = "{}|{}|{}|{}".format(subg, sect, clad, spec)
    else:
        tips_dict04[tipdata04.SRA[idx]] = "{}|{}|{}|Q.{}".format(subg, sect, clad, spec)
ntre04 = tre04.set_node_values('name', tips_dict04)
rtre04 = ntre04.root(wildcard="Lithocarpus")
canvas04, axes04, mark04 = rtre04.draw(
    height=2000,
    tip_labels_style={"font-size": "8px"},
    use_edge_lengths=False,
    edge_colors=rtre04.get_edge_values_mapped({
        rtre04.get_mrca_idx_from_tip_labels(wildcard="Cerris|"),
        rtre04.get_mrca_idx_from_tip_labels(wildcard="Lobatae|"),
        rtre04.get_mrca_idx_from_tip_labels(wildcard="Roburoids|"),
        rtre04.get_mrca_idx_from_tip_labels(wildcard="Virentes|"),
        rtre04.get_mrca_idx_from_tip_labels(wildcard="Protoba"),
        rtre04.get_mrca_idx_from_tip_labels(wildcard="Leuco"),
        rtre04.get_mrca_idx_from_tip_labels(wildcard="Ponticae|"),
    }),
)

<svg class="toyplot-canvas-Canvas" height="2000.0px" id="te20e3397d23f48dd80e44b556637f12d" preserveAspectRatio="xMidYMid meet" style="background-color:transparent;border-color:#292724;border-style:none;border-width:1.0;fill:rgb(16.1%,15.3%,14.1%);fill-opacity:1.0;font-family:Helvetica;font-size:12px;opacity:1.0;stroke:rgb(16.1%,15.3%,14.1%);stroke-opacity:1.0;stroke-width:1.0" viewBox="0 0 500.0 2000.0" width="500.0px" xmlns="http://www.w3.org/2000/svg" xmlns:toyplot="http://www.sandia.gov/toyplot" xmlns:xlink="http://www.w3.org/1999/xlink"> Quercus|Quercus|Leucomexicana|Q.nudinervis Quercus|Quercus|Leucomexicana|Q.magnoliifolia Quercus|Quercus|Leucomexicana|Q.resinosa Quercus|Quercus|Leucomexicana|Q.liebmanii Quercus|Quercus|Leucomexicana|Q.purulhana Quercus|Quercus|Leucomexicana|Q.segoviensis Quercus|Quercus|Leucomexicana|Q.subspathulata Quercus|Quercus|Leucomexicana|Q.species Quercus|Quercus|Leucomexicana|Q.glaucescens Quercus|Quercus|Leucomexicana|Q.potosina Quercus|Quercus|Leucomexicana|Q.deserticola Quercus|Quercus|Leucomexicana|Q.glaucoides Quercus|Quercus|Leucomexicana|Q.laeta Quercus|Quercus|Leucomexicana|Q.peduncularis Quercus|Quercus|Leucomexicana|Q.arizonica Quercus|Quercus|Leucomexicana|Q.obtusata Quercus|Quercus|Leucomexicana|Q.laeta Quercus|Quercus|Leucomexicana|Q.arizonica Quercus|Quercus|Leucomexicana|Q.germana Quercus|Quercus|Leucomexicana|Q.glabrescens Quercus|Quercus|Leucomexicana|Q.oblongifolia Quercus|Quercus|Leucomexicana|Q.chihuahuensis Quercus|Quercus|Leucomexicana|Q.engelmannii Quercus|Quercus|Leucomexicana|Q.ajoensis Quercus|Quercus|Leucomexicana|Q.turbinella Quercus|Quercus|Leucomexicana|Q.toumeyi Quercus|Quercus|Leucomexicana|Q.grisea Quercus|Quercus|Leucomexicana|Q.greggii Quercus|Quercus|Leucomexicana|Q.rugosa Quercus|Quercus|Leucomexicana|Q.striatula Quercus|Quercus|Leucomexicana|Q.diversifolia Quercus|Quercus|Leucomexicana|Q.corrugata Quercus|Quercus|Leucomexicana|Q.lancifolia Quercus|Quercus|Leucomexicana|Q.insignis Quercus|Quercus|Leucomexicana|Q.copeyensis Quercus|Quercus|Leucomexicana|Q.martinezii Quercus|Quercus|Stellatae|Q.boyntonii Quercus|Quercus|Stellatae|Q.similis Quercus|Quercus|Stellatae|Q.stellata Quercus|Quercus|Stellatae|Q.chapmanii Quercus|Quercus|Stellatae|Q.austrina Quercus|Quercus|Stellatae|Q.oglethorpensis Quercus|Quercus|Stellatae|Q.havardii Quercus|Quercus|Stellatae|Q.margarettae Quercus|Quercus|Stellatae|Q.sinuata Quercus|Quercus|Texas white oaks|Q.pungens Quercus|Quercus|Texas white oaks|Q.vaseyana Quercus|Quercus|Stellatae|Q.laceyi Quercus|Quercus|Texas white oaks|Q.hinckleyi Quercus|Quercus|Texas white oaks|Q.polymorpha Quercus|Quercus|Texas white oaks|Q.mohriana Quercus|Quercus|Prinoids|Q.prinoides Quercus|Quercus|Prinoids|Q.muehlenbergii Quercus|Quercus|Prinoids|Q.lyrata Quercus|Quercus|Prinoids|Q.bicolor Quercus|Quercus|Roburoids|Q.macranthera Quercus|Quercus|Roburoids|Q.canariensis Quercus|Quercus|Roburoids|Q.kotschyana Quercus|Quercus|Roburoids|Q.frainetto Quercus|Quercus|Roburoids|Q.pubescens Quercus|Quercus|Roburoids|Q.dalechampii Quercus|Quercus|Roburoids|Q.cedrorum Quercus|Quercus|Roburoids|Q.infectoria Quercus|Quercus|Roburoids|Q.boissieri Quercus|Quercus|Roburoids|Q.pyrenaica Quercus|Quercus|Roburoids|Q.faginea Quercus|Quercus|Roburoids|Q.imeretina Quercus|Quercus|Roburoids|Q.hartwissiana Quercus|Quercus|Roburoids|Q.robur reference Quercus|Quercus|Roburoids|Q.petraea Quercus|Quercus|Roburoids|Q.lusitanica Quercus|Quercus|Roburoids|Q.vulcanica Quercus|Quercus|Roburoids|Q.yunnanensis Quercus|Quercus|Roburoids|Q.mongolica Quercus|Quercus|Roburoids|Q.dentata Quercus|Quercus|Roburoids|Q.aliena Quercus|Quercus|Roburoids|Q.fabri Quercus|Quercus|Roburoids|Q.griffithii Quercus|Quercus|Roburoids|Q.serrata Quercus|Quercus|Albae|Q.montana Quercus|Quercus|Albae|Q.alba Quercus|Quercus|Albae|Q.michauxii Quercus|Quercus|Prinoids|Q.macrocarpa Quercus|Quercus|Dumosae|Q.douglasii Quercus|Quercus|Dumosae|Q.pacifica Quercus|Quercus|Dumosae|Q.john-tuckeri Quercus|Quercus|Dumosae

In [7]:
tre05 = toytree.tree("/moto/eaton/users/hnl2109/analysis-raxml/RAxML_bipartitions.basic_scaffold05")
mask05 = [i in tre05.get_tip_labels() for i in data.SRA]
tipdata05 = data.loc[mask05, :]
tipdata05 = tipdata05.reset_index(drop=True)
tipdata05.loc[tipdata05.subgenus.isna(), ["subgenus", "section", "clade"]] = ""
tips_dict05 = {}
for idx in range(tipdata05.shape[0]):
    if tipdata05.subgenus[idx] == "" :
        subg = sect = clad = "NaN"
        spec = tipdata05.species[idx].replace(" ", "-")
    else:
        subg = tipdata05.subgenus[idx].split()[-1]
        sect = tipdata05.section[idx]
        clad = tipdata05.clade[idx]
        spec = tipdata05.species[idx].split()[-1]
    if tipdata05.subgenus[idx] == "" :
        tips_dict05[tipdata05.SRA[idx]] = "{}|{}|{}|{}".format(subg, sect, clad, spec)
    else:
        tips_dict05[tipdata05.SRA[idx]] = "{}|{}|{}|Q.{}".format(subg, sect, clad, spec)
ntre05 = tre05.set_node_values('name', tips_dict05)
rtre05 = ntre05.root(wildcard="Lithocarpus")
canvas05, axes05, mark05 = rtre05.draw(
    height=2000,
    tip_labels_style={"font-size": "8px"},
    use_edge_lengths=False,
    edge_colors=rtre05.get_edge_values_mapped({
        rtre05.get_mrca_idx_from_tip_labels(wildcard="Cerris|"),
        rtre05.get_mrca_idx_from_tip_labels(wildcard="Lobatae|"),
        rtre05.get_mrca_idx_from_tip_labels(wildcard="Roburoids|"),
        rtre05.get_mrca_idx_from_tip_labels(wildcard="Virentes|"),
        rtre05.get_mrca_idx_from_tip_labels(wildcard="Protoba"),
        rtre05.get_mrca_idx_from_tip_labels(wildcard="Leuco"),
        rtre05.get_mrca_idx_from_tip_labels(wildcard="Ponticae|"),
    }),
)

<svg class="toyplot-canvas-Canvas" height="2000.0px" id="t2c895ce9f2984245ad5c92305a0596fc" preserveAspectRatio="xMidYMid meet" style="background-color:transparent;border-color:#292724;border-style:none;border-width:1.0;fill:rgb(16.1%,15.3%,14.1%);fill-opacity:1.0;font-family:Helvetica;font-size:12px;opacity:1.0;stroke:rgb(16.1%,15.3%,14.1%);stroke-opacity:1.0;stroke-width:1.0" viewBox="0 0 500.0 2000.0" width="500.0px" xmlns="http://www.w3.org/2000/svg" xmlns:toyplot="http://www.sandia.gov/toyplot" xmlns:xlink="http://www.w3.org/1999/xlink"> Quercus|Quercus|Leucomexicana|Q.oblongifolia Quercus|Quercus|Leucomexicana|Q.arizonica Quercus|Quercus|Leucomexicana|Q.laeta Quercus|Quercus|Leucomexicana|Q.laeta Quercus|Quercus|Leucomexicana|Q.arizonica Quercus|Quercus|Leucomexicana|Q.ajoensis Quercus|Quercus|Leucomexicana|Q.turbinella Quercus|Quercus|Leucomexicana|Q.toumeyi Quercus|Quercus|Leucomexicana|Q.chihuahuensis Quercus|Quercus|Leucomexicana|Q.grisea Quercus|Quercus|Leucomexicana|Q.glaucoides Quercus|Quercus|Leucomexicana|Q.diversifolia Quercus|Quercus|Leucomexicana|Q.deserticola Quercus|Quercus|Leucomexicana|Q.peduncularis Quercus|Quercus|Leucomexicana|Q.obtusata Quercus|Quercus|Leucomexicana|Q.greggii Quercus|Quercus|Leucomexicana|Q.rugosa Quercus|Quercus|Leucomexicana|Q.glabrescens Quercus|Quercus|Leucomexicana|Q.striatula Quercus|Quercus|Leucomexicana|Q.potosina Quercus|Quercus|Leucomexicana|Q.nudinervis Quercus|Quercus|Leucomexicana|Q.subspathulata Quercus|Quercus|Leucomexicana|Q.resinosa Quercus|Quercus|Leucomexicana|Q.magnoliifolia Quercus|Quercus|Leucomexicana|Q.glaucescens Quercus|Quercus|Leucomexicana|Q.liebmanii Quercus|Quercus|Leucomexicana|Q.purulhana Quercus|Quercus|Leucomexicana|Q.segoviensis Quercus|Quercus|Leucomexicana|Q.species Quercus|Quercus|Leucomexicana|Q.copeyensis Quercus|Quercus|Leucomexicana|Q.corrugata Quercus|Quercus|Leucomexicana|Q.insignis Quercus|Quercus|Leucomexicana|Q.lancifolia Quercus|Quercus|Leucomexicana|Q.germana Quercus|Quercus|Leucomexicana|Q.martinezii Quercus|Quercus|Stellatae|Q.stellata Quercus|Quercus|Stellatae|Q.boyntonii Quercus|Quercus|Stellatae|Q.similis Quercus|Quercus|Stellatae|Q.chapmanii Quercus|Quercus|Stellatae|Q.margarettae Quercus|Quercus|Stellatae|Q.austrina Quercus|Quercus|Stellatae|Q.oglethorpensis Quercus|Quercus|Stellatae|Q.havardii Quercus|Quercus|Stellatae|Q.sinuata Quercus|Quercus|Texas white oaks|Q.pungens Quercus|Quercus|Texas white oaks|Q.vaseyana Quercus|Quercus|Texas white oaks|Q.hinckleyi Quercus|Quercus|Stellatae|Q.laceyi Quercus|Quercus|Texas white oaks|Q.polymorpha Quercus|Quercus|Texas white oaks|Q.mohriana Quercus|Quercus|Roburoids|Q.kotschyana Quercus|Quercus|Roburoids|Q.dalechampii Quercus|Quercus|Roburoids|Q.frainetto Quercus|Quercus|Roburoids|Q.pubescens Quercus|Quercus|Roburoids|Q.vulcanica Quercus|Quercus|Roburoids|Q.boissieri Quercus|Quercus|Roburoids|Q.infectoria Quercus|Quercus|Roburoids|Q.cedrorum Quercus|Quercus|Roburoids|Q.pyrenaica Quercus|Quercus|Roburoids|Q.macranthera Quercus|Quercus|Roburoids|Q.faginea Quercus|Quercus|Roburoids|Q.petraea reference Quercus|Quercus|Roburoids|Q.canariensis Quercus|Quercus|Roburoids|Q.robur Quercus|Quercus|Roburoids|Q.hartwissiana Quercus|Quercus|Roburoids|Q.lusitanica Quercus|Quercus|Roburoids|Q.imeretina Quercus|Quercus|Roburoids|Q.fabri Quercus|Quercus|Roburoids|Q.aliena Quercus|Quercus|Roburoids|Q.griffithii Quercus|Quercus|Roburoids|Q.yunnanensis Quercus|Quercus|Roburoids|Q.serrata Quercus|Quercus|Roburoids|Q.dentata Quercus|Quercus|Roburoids|Q.mongolica Quercus|Quercus|Albae|Q.alba Quercus|Quercus|Albae|Q.michauxii Quercus|Quercus|Albae|Q.montana Quercus|Quercus|Dumosae|Q.dumosa Quercus|Quercus|Dumosae|Q.pacifica Quercus|Quercus|Dumosae|Q.cornelius-mulleri Quercus|Quercus|Dumosae|Q.john-tuckeri Quercus|Quercus|Dumosae|Q.douglasii Quercus|Quercus|Dumosae|Q.berberidifolia Quercus|Quercus|Dumosae|Q.durata Quercus|Quercus|Leucomexicana|Q.engelmannii Quercus|Quercus|Dumosae|Q.garryana Quercus|Quercus|Dumosa

In [4]:
tre06 = toytree.tree("/moto/eaton/users/hnl2109/analysis-raxml/RAxML_bipartitions.basic_scaffold06")
mask06 = [i in tre06.get_tip_labels() for i in data.SRA]
tipdata06 = data.loc[mask06, :]
tipdata06 = tipdata06.reset_index(drop=True)
tipdata06.loc[tipdata06.subgenus.isna(), ["subgenus", "section", "clade"]] = ""
tips_dict06 = {}
for idx in range(tipdata06.shape[0]):
    if tipdata06.subgenus[idx] == "" :
        subg = sect = clad = "NaN"
        spec = tipdata06.species[idx].replace(" ", "-")
    else:
        subg = tipdata06.subgenus[idx].split()[-1]
        sect = tipdata06.section[idx]
        clad = tipdata06.clade[idx]
        spec = tipdata06.species[idx].split()[-1]
    if tipdata06.subgenus[idx] == "" :
        tips_dict06[tipdata06.SRA[idx]] = "{}|{}|{}|{}".format(subg, sect, clad, spec)
    else:
        tips_dict06[tipdata06.SRA[idx]] = "{}|{}|{}|Q.{}".format(subg, sect, clad, spec)
ntre06 = tre06.set_node_values('name', tips_dict06)
rtre06 = ntre06.root(wildcard="Lithocarpus")
canvas06, axes06, mark06 = rtre06.draw(
    height=2000,
    tip_labels_style={"font-size": "8px"},
    use_edge_lengths=False,
    edge_colors=rtre06.get_edge_values_mapped({
        rtre06.get_mrca_idx_from_tip_labels(wildcard="Cerris|"),
        rtre06.get_mrca_idx_from_tip_labels(wildcard="Lobatae|"),
        rtre06.get_mrca_idx_from_tip_labels(wildcard="Roburoids|"),
        rtre06.get_mrca_idx_from_tip_labels(wildcard="Virentes|"),
        rtre06.get_mrca_idx_from_tip_labels(wildcard="Protoba"),
        rtre06.get_mrca_idx_from_tip_labels(wildcard="Leuco"),
        rtre06.get_mrca_idx_from_tip_labels(wildcard="Ponticae|"),
    }),
)

<svg class="toyplot-canvas-Canvas" height="2000.0px" id="td4349e46039e49be92098f186a490d5e" preserveAspectRatio="xMidYMid meet" style="background-color:transparent;border-color:#292724;border-style:none;border-width:1.0;fill:rgb(16.1%,15.3%,14.1%);fill-opacity:1.0;font-family:Helvetica;font-size:12px;opacity:1.0;stroke:rgb(16.1%,15.3%,14.1%);stroke-opacity:1.0;stroke-width:1.0" viewBox="0 0 500.0 2000.0" width="500.0px" xmlns="http://www.w3.org/2000/svg" xmlns:toyplot="http://www.sandia.gov/toyplot" xmlns:xlink="http://www.w3.org/1999/xlink"> Quercus|Quercus|Stellatae|Q.similis Quercus|Quercus|Stellatae|Q.chapmanii Quercus|Quercus|Stellatae|Q.stellata Quercus|Quercus|Stellatae|Q.boyntonii Quercus|Quercus|Stellatae|Q.margarettae Quercus|Quercus|Stellatae|Q.havardii Quercus|Quercus|Stellatae|Q.austrina Quercus|Quercus|Stellatae|Q.oglethorpensis Quercus|Quercus|Stellatae|Q.sinuata Quercus|Quercus|Stellatae|Q.laceyi Quercus|Quercus|Texas white oaks|Q.hinckleyi Quercus|Quercus|Texas white oaks|Q.pungens Quercus|Quercus|Texas white oaks|Q.vaseyana Quercus|Quercus|Texas white oaks|Q.mohriana Quercus|Quercus|Texas white oaks|Q.polymorpha Quercus|Quercus|Leucomexicana|Q.corrugata Quercus|Quercus|Leucomexicana|Q.insignis Quercus|Quercus|Leucomexicana|Q.martinezii Quercus|Quercus|Leucomexicana|Q.germana Quercus|Quercus|Leucomexicana|Q.glaucescens Quercus|Quercus|Leucomexicana|Q.resinosa Quercus|Quercus|Leucomexicana|Q.subspathulata Quercus|Quercus|Leucomexicana|Q.nudinervis Quercus|Quercus|Leucomexicana|Q.purulhana Quercus|Quercus|Leucomexicana|Q.segoviensis Quercus|Quercus|Leucomexicana|Q.liebmanii Quercus|Quercus|Leucomexicana|Q.magnoliifolia Quercus|Quercus|Leucomexicana|Q.peduncularis Quercus|Quercus|Leucomexicana|Q.species Quercus|Quercus|Leucomexicana|Q.ajoensis Quercus|Quercus|Leucomexicana|Q.turbinella Quercus|Quercus|Leucomexicana|Q.toumeyi Quercus|Quercus|Leucomexicana|Q.deserticola Quercus|Quercus|Leucomexicana|Q.arizonica Quercus|Quercus|Leucomexicana|Q.oblongifolia Quercus|Quercus|Leucomexicana|Q.grisea Quercus|Quercus|Leucomexicana|Q.striatula Quercus|Quercus|Leucomexicana|Q.glaucoides Quercus|Quercus|Leucomexicana|Q.arizonica Quercus|Quercus|Leucomexicana|Q.laeta Quercus|Quercus|Leucomexicana|Q.laeta Quercus|Quercus|Leucomexicana|Q.chihuahuensis Quercus|Quercus|Leucomexicana|Q.diversifolia Quercus|Quercus|Leucomexicana|Q.potosina Quercus|Quercus|Leucomexicana|Q.obtusata Quercus|Quercus|Leucomexicana|Q.greggii Quercus|Quercus|Leucomexicana|Q.rugosa Quercus|Quercus|Leucomexicana|Q.lancifolia Quercus|Quercus|Leucomexicana|Q.copeyensis Quercus|Quercus|Leucomexicana|Q.glabrescens Quercus|Quercus|Dumosae|Q.dumosa Quercus|Quercus|Dumosae|Q.john-tuckeri Quercus|Quercus|Dumosae|Q.cornelius-mulleri Quercus|Quercus|Dumosae|Q.douglasii Quercus|Quercus|Dumosae|Q.pacifica Quercus|Quercus|Dumosae|Q.durata Quercus|Quercus|Dumosae|Q.berberidifolia Quercus|Quercus|Dumosae|Q.garryana Quercus|Quercus|Leucomexicana|Q.engelmannii Quercus|Quercus|Roburoids|Q.griffithii Quercus|Quercus|Roburoids|Q.fabri Quercus|Quercus|Roburoids|Q.yunnanensis Quercus|Quercus|Roburoids|Q.dentata Quercus|Quercus|Roburoids|Q.serrata Quercus|Quercus|Roburoids|Q.aliena Quercus|Quercus|Roburoids|Q.mongolica Quercus|Quercus|Roburoids|Q.canariensis reference Quercus|Quercus|Roburoids|Q.robur Quercus|Quercus|Roburoids|Q.hartwissiana Quercus|Quercus|Roburoids|Q.imeretina Quercus|Quercus|Roburoids|Q.faginea Quercus|Quercus|Roburoids|Q.pyrenaica Quercus|Quercus|Roburoids|Q.kotschyana Quercus|Quercus|Roburoids|Q.vulcanica Quercus|Quercus|Roburoids|Q.macranthera Quercus|Quercus|Roburoids|Q.lusitanica Quercus|Quercus|Roburoids|Q.boissieri Quercus|Quercus|Roburoids|Q.pubescens Quercus|Quercus|Roburoids|Q.infectoria Quercus|Quercus|Roburoids|Q.cedrorum Quercus|Quercus|Roburoids|Q.petraea Quercus|Quercus|Roburoids|Q.frainetto Quercus|Quercus|Roburoids|Q.dalechampii Quercus|Quercus|Prinoids|Q.lyrata Quercus|Quercus|Prinoids|Q.bicolor Quercus|Quercus|Prinoids|Q.macrocarpa Quercus|Qu

In [14]:
tre07 = toytree.tree("/moto/eaton/users/hnl2109/analysis-raxml/RAxML_bipartitions.basic_scaffold07")
mask07 = [i in tre07.get_tip_labels() for i in data.SRA]
tipdata07 = data.loc[mask07, :]
tipdata07 = tipdata07.reset_index(drop=True)
tipdata07.loc[tipdata07.subgenus.isna(), ["subgenus", "section", "clade"]] = ""
tips_dict07 = {}
for idx in range(tipdata07.shape[0]):
    if tipdata07.subgenus[idx] == "" :
        subg = sect = clad = "NaN"
        spec = tipdata07.species[idx].replace(" ", "-")
    else:
        subg = tipdata07.subgenus[idx].split()[-1]
        sect = tipdata07.section[idx]
        clad = tipdata07.clade[idx]
        spec = tipdata07.species[idx].split()[-1]
    if tipdata07.subgenus[idx] == "" :
        tips_dict07[tipdata07.SRA[idx]] = "{}|{}|{}|{}".format(subg, sect, clad, spec)
    else:
        tips_dict07[tipdata07.SRA[idx]] = "{}|{}|{}|Q.{}".format(subg, sect, clad, spec)
ntre07 = tre07.set_node_values('name', tips_dict07)
rtre07 = ntre07.root(wildcard="Lithocarpus")
canvas07, axes07, mark07 = rtre07.draw(
    height=2000,
    tip_labels_style={"font-size": "8px"},
    use_edge_lengths=False,
    edge_colors=rtre07.get_edge_values_mapped({
        rtre07.get_mrca_idx_from_tip_labels(wildcard="Cerris|"),
        rtre07.get_mrca_idx_from_tip_labels(wildcard="Lobatae|"),
        rtre07.get_mrca_idx_from_tip_labels(wildcard="Roburoids|"),
        rtre07.get_mrca_idx_from_tip_labels(wildcard="Virentes|"),
        rtre07.get_mrca_idx_from_tip_labels(wildcard="Protoba"),
        rtre07.get_mrca_idx_from_tip_labels(wildcard="Leuco"),
        rtre07.get_mrca_idx_from_tip_labels(wildcard="Ponticae|"),
    }),
)

<svg class="toyplot-canvas-Canvas" height="2000.0px" id="t28a65d16ce374171b8f6da4e0719ab17" preserveAspectRatio="xMidYMid meet" style="background-color:transparent;border-color:#292724;border-style:none;border-width:1.0;fill:rgb(16.1%,15.3%,14.1%);fill-opacity:1.0;font-family:Helvetica;font-size:12px;opacity:1.0;stroke:rgb(16.1%,15.3%,14.1%);stroke-opacity:1.0;stroke-width:1.0" viewBox="0 0 500.0 2000.0" width="500.0px" xmlns="http://www.w3.org/2000/svg" xmlns:toyplot="http://www.sandia.gov/toyplot" xmlns:xlink="http://www.w3.org/1999/xlink"> Quercus|Quercus|Leucomexicana|Q.ajoensis Quercus|Quercus|Leucomexicana|Q.turbinella Quercus|Quercus|Leucomexicana|Q.grisea Quercus|Quercus|Leucomexicana|Q.toumeyi Quercus|Quercus|Leucomexicana|Q.oblongifolia Quercus|Quercus|Leucomexicana|Q.striatula Quercus|Quercus|Leucomexicana|Q.chihuahuensis Quercus|Quercus|Leucomexicana|Q.glaucoides Quercus|Quercus|Leucomexicana|Q.laeta Quercus|Quercus|Leucomexicana|Q.arizonica Quercus|Quercus|Leucomexicana|Q.laeta Quercus|Quercus|Leucomexicana|Q.arizonica Quercus|Quercus|Leucomexicana|Q.diversifolia Quercus|Quercus|Leucomexicana|Q.greggii Quercus|Quercus|Leucomexicana|Q.potosina Quercus|Quercus|Leucomexicana|Q.obtusata Quercus|Quercus|Leucomexicana|Q.peduncularis Quercus|Quercus|Leucomexicana|Q.deserticola Quercus|Quercus|Leucomexicana|Q.nudinervis Quercus|Quercus|Leucomexicana|Q.subspathulata Quercus|Quercus|Leucomexicana|Q.resinosa Quercus|Quercus|Leucomexicana|Q.magnoliifolia Quercus|Quercus|Leucomexicana|Q.liebmanii Quercus|Quercus|Leucomexicana|Q.purulhana Quercus|Quercus|Leucomexicana|Q.segoviensis Quercus|Quercus|Leucomexicana|Q.species Quercus|Quercus|Leucomexicana|Q.glaucescens Quercus|Quercus|Leucomexicana|Q.glabrescens Quercus|Quercus|Leucomexicana|Q.martinezii Quercus|Quercus|Leucomexicana|Q.rugosa Quercus|Quercus|Leucomexicana|Q.corrugata Quercus|Quercus|Leucomexicana|Q.lancifolia Quercus|Quercus|Leucomexicana|Q.insignis Quercus|Quercus|Leucomexicana|Q.copeyensis Quercus|Quercus|Leucomexicana|Q.germana Quercus|Quercus|Stellatae|Q.austrina Quercus|Quercus|Stellatae|Q.margarettae Quercus|Quercus|Stellatae|Q.similis Quercus|Quercus|Stellatae|Q.havardii Quercus|Quercus|Stellatae|Q.stellata Quercus|Quercus|Stellatae|Q.boyntonii Quercus|Quercus|Stellatae|Q.chapmanii Quercus|Quercus|Stellatae|Q.oglethorpensis Quercus|Quercus|Stellatae|Q.sinuata Quercus|Quercus|Texas white oaks|Q.pungens Quercus|Quercus|Texas white oaks|Q.hinckleyi Quercus|Quercus|Texas white oaks|Q.vaseyana Quercus|Quercus|Stellatae|Q.laceyi Quercus|Quercus|Texas white oaks|Q.mohriana Quercus|Quercus|Texas white oaks|Q.polymorpha Quercus|Quercus|Prinoids|Q.bicolor Quercus|Quercus|Prinoids|Q.lyrata Quercus|Quercus|Prinoids|Q.macrocarpa Quercus|Quercus|Prinoids|Q.muehlenbergii Quercus|Quercus|Prinoids|Q.prinoides Quercus|Quercus|Dumosae|Q.lobata Quercus|Quercus|Dumosae|Q.douglasii Quercus|Quercus|Dumosae|Q.john-tuckeri Quercus|Quercus|Dumosae|Q.cornelius-mulleri Quercus|Quercus|Dumosae|Q.pacifica Quercus|Quercus|Dumosae|Q.dumosa Quercus|Quercus|Dumosae|Q.berberidifolia Quercus|Quercus|Dumosae|Q.durata Quercus|Quercus|Dumosae|Q.garryana Quercus|Quercus|Leucomexicana|Q.engelmannii Quercus|Quercus|Albae|Q.alba Quercus|Quercus|Albae|Q.michauxii Quercus|Quercus|Albae|Q.montana Quercus|Quercus|Roburoids|Q.boissieri Quercus|Quercus|Roburoids|Q.infectoria Quercus|Quercus|Roburoids|Q.cedrorum Quercus|Quercus|Roburoids|Q.pubescens Quercus|Quercus|Roburoids|Q.faginea Quercus|Quercus|Roburoids|Q.kotschyana Quercus|Quercus|Roburoids|Q.macranthera Quercus|Quercus|Roburoids|Q.vulcanica Quercus|Quercus|Roburoids|Q.frainetto Quercus|Quercus|Roburoids|Q.dalechampii Quercus|Quercus|Roburoids|Q.pyrenaica Quercus|Quercus|Roburoids|Q.lusitanica Quercus|Quercus|Roburoids|Q.petraea Quercus|Quercus|Roburoids|Q.robur Quercus|Quercus|Roburoids|Q.canariensis reference Quercus|Quercus|Roburoids|Q.hartwissiana Quercus|Quercus|Roburoids|Q.imeretina Quercus|Quercus|Roburoids|Q.fabri Quercus|Quercus|Roburoids|Q.

In [15]:
tre08 = toytree.tree("/moto/eaton/users/hnl2109/analysis-raxml/RAxML_bipartitions.basic_scaffold08")
mask08 = [i in tre08.get_tip_labels() for i in data.SRA]
tipdata08 = data.loc[mask08, :]
tipdata08 = tipdata08.reset_index(drop=True)
tipdata08.loc[tipdata08.subgenus.isna(), ["subgenus", "section", "clade"]] = ""
tips_dict08 = {}
for idx in range(tipdata08.shape[0]):
    if tipdata08.subgenus[idx] == "" :
        subg = sect = clad = "NaN"
        spec = tipdata08.species[idx].replace(" ", "-")
    else:
        subg = tipdata08.subgenus[idx].split()[-1]
        sect = tipdata08.section[idx]
        clad = tipdata08.clade[idx]
        spec = tipdata08.species[idx].split()[-1]
    if tipdata08.subgenus[idx] == "" :
        tips_dict08[tipdata08.SRA[idx]] = "{}|{}|{}|{}".format(subg, sect, clad, spec)
    else:
        tips_dict08[tipdata08.SRA[idx]] = "{}|{}|{}|Q.{}".format(subg, sect, clad, spec)
ntre08 = tre08.set_node_values('name', tips_dict08)
rtre08 = ntre08.root(wildcard="Lithocarpus")
canvas08, axes08, mark08 = rtre08.draw(
    height=2000,
    tip_labels_style={"font-size": "8px"},
    use_edge_lengths=False,
    edge_colors=rtre08.get_edge_values_mapped({
        rtre08.get_mrca_idx_from_tip_labels(wildcard="Cerris|"),
        rtre08.get_mrca_idx_from_tip_labels(wildcard="Lobatae|"),
        rtre08.get_mrca_idx_from_tip_labels(wildcard="Roburoids|"),
        rtre08.get_mrca_idx_from_tip_labels(wildcard="Virentes|"),
        rtre08.get_mrca_idx_from_tip_labels(wildcard="Protoba"),
        rtre08.get_mrca_idx_from_tip_labels(wildcard="Leuco"),
        rtre08.get_mrca_idx_from_tip_labels(wildcard="Ponticae|"),
    }),
)

<svg class="toyplot-canvas-Canvas" height="2000.0px" id="tfacf8856d80848788c5b750f9e2f5179" preserveAspectRatio="xMidYMid meet" style="background-color:transparent;border-color:#292724;border-style:none;border-width:1.0;fill:rgb(16.1%,15.3%,14.1%);fill-opacity:1.0;font-family:Helvetica;font-size:12px;opacity:1.0;stroke:rgb(16.1%,15.3%,14.1%);stroke-opacity:1.0;stroke-width:1.0" viewBox="0 0 500.0 2000.0" width="500.0px" xmlns="http://www.w3.org/2000/svg" xmlns:toyplot="http://www.sandia.gov/toyplot" xmlns:xlink="http://www.w3.org/1999/xlink"> Quercus|Quercus|Leucomexicana|Q.greggii Quercus|Quercus|Leucomexicana|Q.striatula Quercus|Quercus|Leucomexicana|Q.deserticola Quercus|Quercus|Leucomexicana|Q.arizonica Quercus|Quercus|Leucomexicana|Q.obtusata Quercus|Quercus|Leucomexicana|Q.martinezii Quercus|Quercus|Leucomexicana|Q.segoviensis Quercus|Quercus|Leucomexicana|Q.glabrescens Quercus|Quercus|Leucomexicana|Q.toumeyi Quercus|Quercus|Leucomexicana|Q.diversifolia Quercus|Quercus|Leucomexicana|Q.peduncularis Quercus|Quercus|Leucomexicana|Q.magnoliifolia Quercus|Quercus|Leucomexicana|Q.resinosa Quercus|Quercus|Leucomexicana|Q.species Quercus|Quercus|Leucomexicana|Q.nudinervis Quercus|Quercus|Leucomexicana|Q.subspathulata Quercus|Quercus|Leucomexicana|Q.liebmanii Quercus|Quercus|Leucomexicana|Q.purulhana Quercus|Quercus|Leucomexicana|Q.glaucoides Quercus|Quercus|Leucomexicana|Q.oblongifolia Quercus|Quercus|Leucomexicana|Q.laeta Quercus|Quercus|Leucomexicana|Q.laeta Quercus|Quercus|Leucomexicana|Q.arizonica Quercus|Quercus|Leucomexicana|Q.potosina Quercus|Quercus|Leucomexicana|Q.grisea Quercus|Quercus|Leucomexicana|Q.turbinella Quercus|Quercus|Leucomexicana|Q.ajoensis Quercus|Quercus|Leucomexicana|Q.chihuahuensis Quercus|Quercus|Leucomexicana|Q.rugosa Quercus|Quercus|Leucomexicana|Q.corrugata Quercus|Quercus|Leucomexicana|Q.lancifolia Quercus|Quercus|Leucomexicana|Q.glaucescens Quercus|Quercus|Leucomexicana|Q.insignis Quercus|Quercus|Leucomexicana|Q.copeyensis Quercus|Quercus|Leucomexicana|Q.germana Quercus|Quercus|Texas white oaks|Q.pungens Quercus|Quercus|Texas white oaks|Q.hinckleyi Quercus|Quercus|Texas white oaks|Q.vaseyana Quercus|Quercus|Stellatae|Q.laceyi Quercus|Quercus|Texas white oaks|Q.mohriana Quercus|Quercus|Texas white oaks|Q.polymorpha Quercus|Quercus|Stellatae|Q.boyntonii Quercus|Quercus|Stellatae|Q.stellata Quercus|Quercus|Stellatae|Q.similis Quercus|Quercus|Stellatae|Q.havardii Quercus|Quercus|Stellatae|Q.margarettae Quercus|Quercus|Stellatae|Q.austrina Quercus|Quercus|Stellatae|Q.oglethorpensis Quercus|Quercus|Stellatae|Q.chapmanii Quercus|Quercus|Stellatae|Q.sinuata SRR1915533 SRR1915548 SRR1915549 SRR1915534 Quercus|Virentes|nan|Q.minima SRR1915542 SRR1915528 Quercus|Virentes|nan|Q.sagraeana SRR1915539 SRR1915525 Quercus|Virentes|nan|Q.fusiformis Quercus|Quercus|Prinoids|Q.bicolor Quercus|Quercus|Prinoids|Q.lyrata Quercus|Quercus|Prinoids|Q.macrocarpa Quercus|Quercus|Dumosae|Q.lobata Quercus|Quercus|Prinoids|Q.muehlenbergii Quercus|Quercus|Prinoids|Q.prinoides Quercus|Quercus|Dumosae|Q.douglasii Quercus|Quercus|Dumosae|Q.cornelius-mulleri Quercus|Quercus|Dumosae|Q.john-tuckeri Quercus|Quercus|Dumosae|Q.dumosa Quercus|Quercus|Dumosae|Q.pacifica Quercus|Quercus|Dumosae|Q.durata Quercus|Quercus|Dumosae|Q.berberidifolia Quercus|Quercus|Dumosae|Q.garryana Quercus|Quercus|Leucomexicana|Q.engelmannii Quercus|Quercus|Albae|Q.alba Quercus|Quercus|Albae|Q.michauxii Quercus|Quercus|Albae|Q.montana Quercus|Quercus|Roburoids|Q.canariensis Quercus|Quercus|Roburoids|Q.robur Quercus|Quercus|Roburoids|Q.petraea Quercus|Quercus|Roburoids|Q.dalechampii Quercus|Quercus|Roburoids|Q.hartwissiana reference Quercus|Quercus|Roburoids|Q.imeretina Quercus|Quercus|Roburoids|Q.lusitanica Quercus|Quercus|Roburoids|Q.pyrenaica Quercus|Quercus|Roburoids|Q.cedrorum Quercus|Quercus|Roburoids|Q.infectoria Quercus|Quercus|Roburoids|Q.boissieri Quercus|Quercus|Roburoids|Q.kotschyana Quercus|Quercus|Roburoids|Q.faginea Quercus|Quercus|Roburoids|Q.vul

In [16]:
tre09 = toytree.tree("/moto/eaton/users/hnl2109/analysis-raxml/RAxML_bipartitions.basic_scaffold09")
mask09 = [i in tre09.get_tip_labels() for i in data.SRA]
tipdata09 = data.loc[mask09, :]
tipdata09 = tipdata09.reset_index(drop=True)
tipdata09.loc[tipdata09.subgenus.isna(), ["subgenus", "section", "clade"]] = ""
tips_dict09 = {}
for idx in range(tipdata09.shape[0]):
    if tipdata09.subgenus[idx] == "" :
        subg = sect = clad = "NaN"
        spec = tipdata09.species[idx].replace(" ", "-")
    else:
        subg = tipdata09.subgenus[idx].split()[-1]
        sect = tipdata09.section[idx]
        clad = tipdata09.clade[idx]
        spec = tipdata09.species[idx].split()[-1]
    if tipdata09.subgenus[idx] == "" :
        tips_dict09[tipdata09.SRA[idx]] = "{}|{}|{}|{}".format(subg, sect, clad, spec)
    else:
        tips_dict09[tipdata09.SRA[idx]] = "{}|{}|{}|Q.{}".format(subg, sect, clad, spec)
ntre09 = tre09.set_node_values('name', tips_dict09)
rtre09 = ntre09.root(wildcard="Lithocarpus")
canvas09, axes09, mark09 = rtre09.draw(
    height=2000,
    tip_labels_style={"font-size": "8px"},
    use_edge_lengths=False,
    edge_colors=rtre09.get_edge_values_mapped({
        rtre09.get_mrca_idx_from_tip_labels(wildcard="Cerris|"),
        rtre09.get_mrca_idx_from_tip_labels(wildcard="Lobatae|"),
        rtre09.get_mrca_idx_from_tip_labels(wildcard="Roburoids|"),
        rtre09.get_mrca_idx_from_tip_labels(wildcard="Virentes|"),
        rtre09.get_mrca_idx_from_tip_labels(wildcard="Protoba"),
        rtre09.get_mrca_idx_from_tip_labels(wildcard="Leuco"),
        rtre09.get_mrca_idx_from_tip_labels(wildcard="Ponticae|"),
    }),
)

<svg class="toyplot-canvas-Canvas" height="2000.0px" id="tce5b5d00e3fe4234b1c7f96828c3d04f" preserveAspectRatio="xMidYMid meet" style="background-color:transparent;border-color:#292724;border-style:none;border-width:1.0;fill:rgb(16.1%,15.3%,14.1%);fill-opacity:1.0;font-family:Helvetica;font-size:12px;opacity:1.0;stroke:rgb(16.1%,15.3%,14.1%);stroke-opacity:1.0;stroke-width:1.0" viewBox="0 0 500.0 2000.0" width="500.0px" xmlns="http://www.w3.org/2000/svg" xmlns:toyplot="http://www.sandia.gov/toyplot" xmlns:xlink="http://www.w3.org/1999/xlink"> Quercus|Quercus|Stellatae|Q.similis Quercus|Quercus|Stellatae|Q.chapmanii Quercus|Quercus|Stellatae|Q.margarettae Quercus|Quercus|Stellatae|Q.havardii Quercus|Quercus|Stellatae|Q.austrina Quercus|Quercus|Stellatae|Q.stellata Quercus|Quercus|Stellatae|Q.boyntonii Quercus|Quercus|Stellatae|Q.sinuata Quercus|Quercus|Stellatae|Q.oglethorpensis Quercus|Quercus|Texas white oaks|Q.hinckleyi Quercus|Quercus|Texas white oaks|Q.polymorpha Quercus|Quercus|Texas white oaks|Q.vaseyana Quercus|Quercus|Texas white oaks|Q.pungens Quercus|Quercus|Texas white oaks|Q.mohriana Quercus|Quercus|Stellatae|Q.laceyi Quercus|Quercus|Leucomexicana|Q.germana Quercus|Quercus|Leucomexicana|Q.subspathulata Quercus|Quercus|Leucomexicana|Q.nudinervis Quercus|Quercus|Leucomexicana|Q.resinosa Quercus|Quercus|Leucomexicana|Q.magnoliifolia Quercus|Quercus|Leucomexicana|Q.species Quercus|Quercus|Leucomexicana|Q.liebmanii Quercus|Quercus|Leucomexicana|Q.segoviensis Quercus|Quercus|Leucomexicana|Q.purulhana Quercus|Quercus|Leucomexicana|Q.glaucescens Quercus|Quercus|Leucomexicana|Q.lancifolia Quercus|Quercus|Leucomexicana|Q.copeyensis Quercus|Quercus|Leucomexicana|Q.corrugata Quercus|Quercus|Leucomexicana|Q.insignis Quercus|Quercus|Leucomexicana|Q.arizonica Quercus|Quercus|Leucomexicana|Q.arizonica Quercus|Quercus|Leucomexicana|Q.obtusata Quercus|Quercus|Leucomexicana|Q.peduncularis Quercus|Quercus|Leucomexicana|Q.deserticola Quercus|Quercus|Leucomexicana|Q.potosina Quercus|Quercus|Leucomexicana|Q.diversifolia Quercus|Quercus|Leucomexicana|Q.oblongifolia Quercus|Quercus|Leucomexicana|Q.grisea Quercus|Quercus|Leucomexicana|Q.chihuahuensis Quercus|Quercus|Leucomexicana|Q.laeta Quercus|Quercus|Leucomexicana|Q.glaucoides Quercus|Quercus|Leucomexicana|Q.toumeyi Quercus|Quercus|Leucomexicana|Q.ajoensis Quercus|Quercus|Leucomexicana|Q.turbinella Quercus|Quercus|Leucomexicana|Q.striatula Quercus|Quercus|Leucomexicana|Q.rugosa Quercus|Quercus|Leucomexicana|Q.martinezii Quercus|Quercus|Leucomexicana|Q.greggii Quercus|Quercus|Leucomexicana|Q.glabrescens Quercus|Quercus|Leucomexicana|Q.laeta Quercus|Quercus|Leucomexicana|Q.engelmannii Quercus|Quercus|Roburoids|Q.aliena Quercus|Quercus|Roburoids|Q.griffithii Quercus|Quercus|Roburoids|Q.serrata Quercus|Quercus|Roburoids|Q.yunnanensis Quercus|Quercus|Roburoids|Q.dentata Quercus|Quercus|Roburoids|Q.mongolica Quercus|Quercus|Roburoids|Q.fabri Quercus|Quercus|Roburoids|Q.dalechampii Quercus|Quercus|Roburoids|Q.robur reference Quercus|Quercus|Roburoids|Q.canariensis Quercus|Quercus|Roburoids|Q.hartwissiana Quercus|Quercus|Roburoids|Q.imeretina Quercus|Quercus|Roburoids|Q.lusitanica Quercus|Quercus|Roburoids|Q.macranthera Quercus|Quercus|Roburoids|Q.pubescens Quercus|Quercus|Roburoids|Q.frainetto Quercus|Quercus|Roburoids|Q.pyrenaica Quercus|Quercus|Roburoids|Q.petraea Quercus|Quercus|Roburoids|Q.vulcanica Quercus|Quercus|Roburoids|Q.faginea Quercus|Quercus|Roburoids|Q.boissieri Quercus|Quercus|Roburoids|Q.infectoria Quercus|Quercus|Roburoids|Q.kotschyana Quercus|Quercus|Roburoids|Q.cedrorum Quercus|Quercus|Albae|Q.montana Quercus|Quercus|Dumosae|Q.dumosa Quercus|Quercus|Dumosae|Q.pacifica Quercus|Quercus|Dumosae|Q.cornelius-mulleri Quercus|Quercus|Dumosae|Q.douglasii Quercus|Quercus|Dumosae|Q.john-tuckeri Quercus|Quercus|Dumosae|Q.berberidifolia Quercus|Quercus|Dumosae|Q.garryana Quercus|Quercus|Dumosae|Q.durata Quercus|Quercus|Dumosae|Q.lobata Quercus|Quercus|Prinoids|Q.bicolor Quercus|Quercus|P

In [17]:
tre10 = toytree.tree("/moto/eaton/users/hnl2109/analysis-raxml/RAxML_bipartitions.basic_scaffold10")
mask10 = [i in tre10.get_tip_labels() for i in data.SRA]
tipdata10 = data.loc[mask10, :]
tipdata10 = tipdata10.reset_index(drop=True)
tipdata10.loc[tipdata10.subgenus.isna(), ["subgenus", "section", "clade"]] = ""
tips_dict10 = {}
for idx in range(tipdata10.shape[0]):
    if tipdata10.subgenus[idx] == "" :
        subg = sect = clad = "NaN"
        spec = tipdata10.species[idx].replace(" ", "-")
    else:
        subg = tipdata10.subgenus[idx].split()[-1]
        sect = tipdata10.section[idx]
        clad = tipdata10.clade[idx]
        spec = tipdata10.species[idx].split()[-1]
    if tipdata10.subgenus[idx] == "" :
        tips_dict10[tipdata10.SRA[idx]] = "{}|{}|{}|{}".format(subg, sect, clad, spec)
    else:
        tips_dict10[tipdata10.SRA[idx]] = "{}|{}|{}|Q.{}".format(subg, sect, clad, spec)
ntre10 = tre10.set_node_values('name', tips_dict10)
rtre10 = ntre10.root(wildcard="Lithocarpus")
canvas10, axes10, mark10 = rtre10.draw(
    height=2000,
    tip_labels_style={"font-size": "8px"},
    use_edge_lengths=False,
    edge_colors=rtre10.get_edge_values_mapped({
        rtre10.get_mrca_idx_from_tip_labels(wildcard="Cerris|"),
        rtre10.get_mrca_idx_from_tip_labels(wildcard="Lobatae|"),
        rtre10.get_mrca_idx_from_tip_labels(wildcard="Roburoids|"),
        rtre10.get_mrca_idx_from_tip_labels(wildcard="Virentes|"),
        rtre10.get_mrca_idx_from_tip_labels(wildcard="Protoba"),
        rtre10.get_mrca_idx_from_tip_labels(wildcard="Leuco"),
        rtre10.get_mrca_idx_from_tip_labels(wildcard="Ponticae|"),
    }),
)

<svg class="toyplot-canvas-Canvas" height="2000.0px" id="t08f45a8252bd4f1c8b7a73ff7f525646" preserveAspectRatio="xMidYMid meet" style="background-color:transparent;border-color:#292724;border-style:none;border-width:1.0;fill:rgb(16.1%,15.3%,14.1%);fill-opacity:1.0;font-family:Helvetica;font-size:12px;opacity:1.0;stroke:rgb(16.1%,15.3%,14.1%);stroke-opacity:1.0;stroke-width:1.0" viewBox="0 0 500.0 2000.0" width="500.0px" xmlns="http://www.w3.org/2000/svg" xmlns:toyplot="http://www.sandia.gov/toyplot" xmlns:xlink="http://www.w3.org/1999/xlink"> Quercus|Quercus|Leucomexicana|Q.insignis Quercus|Quercus|Leucomexicana|Q.corrugata Quercus|Quercus|Leucomexicana|Q.copeyensis Quercus|Quercus|Leucomexicana|Q.glaucescens Quercus|Quercus|Leucomexicana|Q.lancifolia Quercus|Quercus|Leucomexicana|Q.glabrescens Quercus|Quercus|Leucomexicana|Q.martinezii Quercus|Quercus|Leucomexicana|Q.rugosa Quercus|Quercus|Leucomexicana|Q.obtusata Quercus|Quercus|Leucomexicana|Q.potosina Quercus|Quercus|Leucomexicana|Q.glaucoides Quercus|Quercus|Leucomexicana|Q.striatula Quercus|Quercus|Leucomexicana|Q.laeta Quercus|Quercus|Leucomexicana|Q.arizonica Quercus|Quercus|Leucomexicana|Q.deserticola Quercus|Quercus|Leucomexicana|Q.arizonica Quercus|Quercus|Leucomexicana|Q.turbinella Quercus|Quercus|Leucomexicana|Q.ajoensis Quercus|Quercus|Leucomexicana|Q.grisea Quercus|Quercus|Leucomexicana|Q.toumeyi Quercus|Quercus|Leucomexicana|Q.oblongifolia Quercus|Quercus|Leucomexicana|Q.chihuahuensis Quercus|Quercus|Leucomexicana|Q.diversifolia Quercus|Quercus|Leucomexicana|Q.laeta Quercus|Quercus|Leucomexicana|Q.segoviensis Quercus|Quercus|Leucomexicana|Q.purulhana Quercus|Quercus|Leucomexicana|Q.nudinervis Quercus|Quercus|Leucomexicana|Q.species Quercus|Quercus|Leucomexicana|Q.magnoliifolia Quercus|Quercus|Leucomexicana|Q.resinosa Quercus|Quercus|Leucomexicana|Q.subspathulata Quercus|Quercus|Leucomexicana|Q.liebmanii Quercus|Quercus|Leucomexicana|Q.peduncularis Quercus|Quercus|Leucomexicana|Q.greggii Quercus|Quercus|Leucomexicana|Q.germana Quercus|Quercus|Stellatae|Q.boyntonii Quercus|Quercus|Stellatae|Q.chapmanii Quercus|Quercus|Stellatae|Q.stellata Quercus|Quercus|Stellatae|Q.oglethorpensis Quercus|Quercus|Stellatae|Q.similis Quercus|Quercus|Stellatae|Q.margarettae Quercus|Quercus|Stellatae|Q.austrina Quercus|Quercus|Stellatae|Q.havardii Quercus|Quercus|Stellatae|Q.sinuata Quercus|Quercus|Texas white oaks|Q.pungens Quercus|Quercus|Texas white oaks|Q.vaseyana Quercus|Quercus|Texas white oaks|Q.mohriana Quercus|Quercus|Stellatae|Q.laceyi Quercus|Quercus|Texas white oaks|Q.polymorpha Quercus|Quercus|Texas white oaks|Q.hinckleyi Quercus|Quercus|Prinoids|Q.muehlenbergii Quercus|Quercus|Prinoids|Q.prinoides Quercus|Quercus|Albae|Q.alba Quercus|Quercus|Albae|Q.michauxii Quercus|Quercus|Prinoids|Q.macrocarpa Quercus|Quercus|Prinoids|Q.bicolor Quercus|Quercus|Prinoids|Q.lyrata Quercus|Quercus|Albae|Q.montana Quercus|Quercus|Dumosae|Q.douglasii Quercus|Quercus|Dumosae|Q.john-tuckeri Quercus|Quercus|Dumosae|Q.pacifica Quercus|Quercus|Dumosae|Q.cornelius-mulleri Quercus|Quercus|Dumosae|Q.dumosa Quercus|Quercus|Dumosae|Q.durata Quercus|Quercus|Leucomexicana|Q.engelmannii Quercus|Quercus|Dumosae|Q.berberidifolia Quercus|Quercus|Dumosae|Q.lobata Quercus|Quercus|Dumosae|Q.garryana Quercus|Quercus|Roburoids|Q.imeretina Quercus|Quercus|Roburoids|Q.canariensis Quercus|Quercus|Roburoids|Q.hartwissiana Quercus|Quercus|Roburoids|Q.robur Quercus|Quercus|Roburoids|Q.lusitanica Quercus|Quercus|Roburoids|Q.faginea Quercus|Quercus|Roburoids|Q.macranthera Quercus|Quercus|Roburoids|Q.vulcanica Quercus|Quercus|Roburoids|Q.petraea Quercus|Quercus|Roburoids|Q.pubescens Quercus|Quercus|Roburoids|Q.frainetto Quercus|Quercus|Roburoids|Q.infectoria Quercus|Quercus|Roburoids|Q.boissieri Quercus|Quercus|Roburoids|Q.cedrorum Quercus|Quercus|Roburoids|Q.kotschyana Quercus|Quercus|Roburoids|Q.dalechampii Quercus|Quercus|Roburoids|Q.pyrenaica reference Quercus|Quercus|Roburoids|Q.aliena Quercus|Quercus|Roburoids|Q

In [18]:
tre11 = toytree.tree("/moto/eaton/users/hnl2109/analysis-raxml/RAxML_bipartitions.basic_scaffold11")
mask11 = [i in tre11.get_tip_labels() for i in data.SRA]
tipdata11 = data.loc[mask11, :]
tipdata11 = tipdata11.reset_index(drop=True)
tipdata11.loc[tipdata11.subgenus.isna(), ["subgenus", "section", "clade"]] = ""
tips_dict11 = {}
for idx in range(tipdata11.shape[0]):
    if tipdata11.subgenus[idx] == "" :
        subg = sect = clad = "NaN"
        spec = tipdata11.species[idx].replace(" ", "-")
    else:
        subg = tipdata11.subgenus[idx].split()[-1]
        sect = tipdata11.section[idx]
        clad = tipdata11.clade[idx]
        spec = tipdata11.species[idx].split()[-1]
    if tipdata11.subgenus[idx] == "" :
        tips_dict11[tipdata11.SRA[idx]] = "{}|{}|{}|{}".format(subg, sect, clad, spec)
    else:
        tips_dict11[tipdata11.SRA[idx]] = "{}|{}|{}|Q.{}".format(subg, sect, clad, spec)
ntre11 = tre11.set_node_values('name', tips_dict11)
rtre11 = ntre11.root(wildcard="Lithocarpus")
canvas11, axes11, mark11 = rtre11.draw(
    height=2000,
    tip_labels_style={"font-size": "8px"},
    use_edge_lengths=False,
    edge_colors=rtre11.get_edge_values_mapped({
        rtre11.get_mrca_idx_from_tip_labels(wildcard="Cerris|"),
        rtre11.get_mrca_idx_from_tip_labels(wildcard="Lobatae|"),
        rtre11.get_mrca_idx_from_tip_labels(wildcard="Roburoids|"),
        rtre11.get_mrca_idx_from_tip_labels(wildcard="Virentes|"),
        rtre11.get_mrca_idx_from_tip_labels(wildcard="Protoba"),
        rtre11.get_mrca_idx_from_tip_labels(wildcard="Leuco"),
        rtre11.get_mrca_idx_from_tip_labels(wildcard="Ponticae|"),
    }),
)

<svg class="toyplot-canvas-Canvas" height="2000.0px" id="t8b24791a94f34dbabcc107b76652ce9a" preserveAspectRatio="xMidYMid meet" style="background-color:transparent;border-color:#292724;border-style:none;border-width:1.0;fill:rgb(16.1%,15.3%,14.1%);fill-opacity:1.0;font-family:Helvetica;font-size:12px;opacity:1.0;stroke:rgb(16.1%,15.3%,14.1%);stroke-opacity:1.0;stroke-width:1.0" viewBox="0 0 500.0 2000.0" width="500.0px" xmlns="http://www.w3.org/2000/svg" xmlns:toyplot="http://www.sandia.gov/toyplot" xmlns:xlink="http://www.w3.org/1999/xlink"> Quercus|Quercus|Leucomexicana|Q.species Quercus|Quercus|Leucomexicana|Q.glaucescens Quercus|Quercus|Leucomexicana|Q.segoviensis Quercus|Quercus|Leucomexicana|Q.purulhana Quercus|Quercus|Leucomexicana|Q.liebmanii Quercus|Quercus|Leucomexicana|Q.resinosa Quercus|Quercus|Leucomexicana|Q.nudinervis Quercus|Quercus|Leucomexicana|Q.magnoliifolia Quercus|Quercus|Leucomexicana|Q.subspathulata Quercus|Quercus|Leucomexicana|Q.peduncularis Quercus|Quercus|Leucomexicana|Q.diversifolia Quercus|Quercus|Leucomexicana|Q.potosina Quercus|Quercus|Leucomexicana|Q.obtusata Quercus|Quercus|Leucomexicana|Q.laeta Quercus|Quercus|Leucomexicana|Q.deserticola Quercus|Quercus|Leucomexicana|Q.arizonica Quercus|Quercus|Leucomexicana|Q.glaucoides Quercus|Quercus|Leucomexicana|Q.laeta Quercus|Quercus|Leucomexicana|Q.chihuahuensis Quercus|Quercus|Leucomexicana|Q.grisea Quercus|Quercus|Leucomexicana|Q.oblongifolia Quercus|Quercus|Leucomexicana|Q.rugosa Quercus|Quercus|Leucomexicana|Q.greggii Quercus|Quercus|Leucomexicana|Q.turbinella Quercus|Quercus|Leucomexicana|Q.ajoensis Quercus|Quercus|Leucomexicana|Q.toumeyi Quercus|Quercus|Leucomexicana|Q.arizonica Quercus|Quercus|Leucomexicana|Q.striatula Quercus|Quercus|Texas white oaks|Q.pungens Quercus|Quercus|Texas white oaks|Q.hinckleyi Quercus|Quercus|Texas white oaks|Q.vaseyana Quercus|Quercus|Stellatae|Q.laceyi Quercus|Quercus|Texas white oaks|Q.mohriana Quercus|Quercus|Texas white oaks|Q.polymorpha Quercus|Quercus|Leucomexicana|Q.insignis Quercus|Quercus|Leucomexicana|Q.corrugata Quercus|Quercus|Leucomexicana|Q.lancifolia Quercus|Quercus|Leucomexicana|Q.copeyensis Quercus|Quercus|Leucomexicana|Q.martinezii Quercus|Quercus|Leucomexicana|Q.germana Quercus|Quercus|Leucomexicana|Q.glabrescens Quercus|Quercus|Stellatae|Q.stellata Quercus|Quercus|Stellatae|Q.boyntonii Quercus|Quercus|Stellatae|Q.chapmanii Quercus|Quercus|Stellatae|Q.austrina Quercus|Quercus|Stellatae|Q.havardii Quercus|Quercus|Stellatae|Q.similis Quercus|Quercus|Stellatae|Q.oglethorpensis Quercus|Quercus|Stellatae|Q.margarettae Quercus|Quercus|Stellatae|Q.sinuata Quercus|Quercus|Roburoids|Q.lusitanica Quercus|Quercus|Roburoids|Q.macranthera Quercus|Quercus|Roburoids|Q.petraea Quercus|Quercus|Roburoids|Q.faginea Quercus|Quercus|Roburoids|Q.vulcanica Quercus|Quercus|Roburoids|Q.kotschyana Quercus|Quercus|Roburoids|Q.boissieri Quercus|Quercus|Roburoids|Q.cedrorum Quercus|Quercus|Roburoids|Q.infectoria Quercus|Quercus|Roburoids|Q.pyrenaica Quercus|Quercus|Roburoids|Q.dalechampii Quercus|Quercus|Roburoids|Q.frainetto Quercus|Quercus|Roburoids|Q.pubescens Quercus|Quercus|Roburoids|Q.robur reference Quercus|Quercus|Roburoids|Q.hartwissiana Quercus|Quercus|Roburoids|Q.imeretina Quercus|Quercus|Roburoids|Q.canariensis Quercus|Quercus|Roburoids|Q.griffithii Quercus|Quercus|Roburoids|Q.aliena Quercus|Quercus|Roburoids|Q.fabri Quercus|Quercus|Roburoids|Q.mongolica Quercus|Quercus|Roburoids|Q.serrata Quercus|Quercus|Roburoids|Q.dentata Quercus|Quercus|Roburoids|Q.yunnanensis Quercus|Quercus|Albae|Q.montana Quercus|Quercus|Albae|Q.michauxii Quercus|Quercus|Albae|Q.alba Quercus|Quercus|Prinoids|Q.lyrata Quercus|Quercus|Prinoids|Q.bicolor Quercus|Quercus|Dumosae|Q.lobata Quercus|Quercus|Prinoids|Q.macrocarpa Quercus|Quercus|Prinoids|Q.muehlenbergii Quercus|Quercus|Prinoids|Q.prinoides Quercus|Quercus|Dumosae|Q.pacifica Quercus|Quercus|Dumosae|Q.dumosa Quercus|Quercus|Dumosae|Q.cornelius-mulleri Quercus|Quercus|Dumosae|Q.dougla

In [4]:
rtre06.ntips

261

In [20]:
# Render each tree as an SVG file.
import toyplot.svg
toyplot.svg.render(canvas00, "/moto/eaton/users/hnl2109/svg-trees/basic_scaffold00.svg")
toyplot.svg.render(canvas01, "/moto/eaton/users/hnl2109/svg-trees/basic_scaffold01.svg")
toyplot.svg.render(canvas02, "/moto/eaton/users/hnl2109/svg-trees/basic_scaffold02.svg")
toyplot.svg.render(canvas03, "/moto/eaton/users/hnl2109/svg-trees/basic_scaffold03.svg")
toyplot.svg.render(canvas04, "/moto/eaton/users/hnl2109/svg-trees/basic_scaffold04.svg")
toyplot.svg.render(canvas05, "/moto/eaton/users/hnl2109/svg-trees/basic_scaffold05.svg")
toyplot.svg.render(canvas06, "/moto/eaton/users/hnl2109/svg-trees/basic_scaffold06.svg")
toyplot.svg.render(canvas07, "/moto/eaton/users/hnl2109/svg-trees/basic_scaffold07.svg")
toyplot.svg.render(canvas08, "/moto/eaton/users/hnl2109/svg-trees/basic_scaffold08.svg")
toyplot.svg.render(canvas09, "/moto/eaton/users/hnl2109/svg-trees/basic_scaffold09.svg")
toyplot.svg.render(canvas10, "/moto/eaton/users/hnl2109/svg-trees/basic_scaffold10.svg")
toyplot.svg.render(canvas11, "/moto/eaton/users/hnl2109/svg-trees/basic_scaffold11.svg")

In [6]:
# For penalized likelihood tests in R: find the node indices used by Hipp et al. for fossil calibrations.
print("Crown calibration nodes:")
print(rtre06.get_mrca_idx_from_tip_labels(wildcard = "Q."))
print(rtre06.get_mrca_idx_from_tip_labels(names = ['Quercus|Lobatae|Agrifoliae|Q.agrifolia',
                                                 'Quercus|Lobatae|Erythromexicana|Q.emoryi']))
print(rtre06.get_mrca_idx_from_tip_labels(names = ['SRR5984311',
                                                 'Cerris|Cyclobalanopsis|Acuta|Q.acuta']))
print(rtre06.get_mrca_idx_from_tip_labels(names = ['Quercus|Quercus|Dumosae|Q.lobata',
                                                 'Quercus|Quercus|Leucomexicana|Q.arizonica']))
print(rtre06.get_mrca_idx_from_tip_labels(names = ['Cerris|Ilex|Early-diverging Ilex|Q.franchetii',
                                                 'Cerris|Ilex|Himalayan subalpine|Q.gujavifolia'])) # Alt. from same clade, original has no SRA number?
print(rtre06.get_mrca_idx_from_tip_labels(names = ['Cerris|Ilex|Himalayan subalpine|Q.gujavifolia',
                                                 'Cerris|Ilex|Himalayan subalpine|Q.semecarpifolia']))
print(rtre06.get_mrca_idx_from_tip_labels(names = ['Cerris|Cerris|East Asian Cerris|Q.chenii',
                                                 'Cerris|Cerris|East Asian Cerris|Q.acutissima']))
print(rtre06.get_mrca_idx_from_tip_labels(names = ['Cerris|Cerris|West Eurasian Cerris|Q.crenata',
                                                 'Cerris|Cerris|West Eurasian Cerris|Q.cerris']))

print("Stem calibration nodes:")
print(rtre06.get_mrca_idx_from_tip_labels(names = ['Cerris|Cerris|East Asian Cerris|Q.chenii',
                                                 'NaN|NaN|NaN|Notholithocarpus-densiflorus']))
print(rtre06.get_mrca_idx_from_tip_labels(names = ['Quercus|Lobatae|Agrifoliae|Q.agrifolia',
                                                   'Quercus|Quercus|Leucomexicana|Q.arizonica']))
print(rtre06.get_mrca_idx_from_tip_labels(names = ['SRR5984311','Cerris|Ilex|Himalayan subalpine|Q.gujavifolia']))
print(rtre06.get_mrca_idx_from_tip_labels(names = ['Quercus|Ponticae|nan|Q.pontica',
                                                 'Quercus|Quercus|Leucomexicana|Q.arizonica']))
print(rtre06.get_mrca_idx_from_tip_labels(names = ['Cerris|Ilex|Early-diverging Ilex|Q.franchetii',
                                                 'Cerris|Cerris|West Eurasian Cerris|Q.cerris']))

Crown calibration nodes:
511
506
507
470
503
450
496
495
Stem calibration nodes:
512
509
510
497
508
